In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
plt.plot(np.arange(0, 1, 0.1), 2*np.arange(0, 1, 0.1))

In [ ]:
import h5py

In [ ]:
from scipy.stats import norm

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn, optim
import scipy.io as sio
# import pandas as pd
import datetime
import os
# import readligo as rl
# from gwpy.timeseries import TimeSeries
import math
import random

import copy

In [ ]:
os.chdir('D:\\OneDrive - HKUST Connect\Research\GWNMMAD\GWAD\Codes')

In [ ]:
import torch.nn.functional as F

In [ ]:
epochs_wsl = 100;

In [ ]:
epochs = 100
rTrain = 0.7;
rTest = 0.2;
# input_vector_length = 100
batch_size = 32
num_bins = 40
coef_delta = 0

# 4-AE + WSL

## Networks

In [ ]:
class AutoEncoder_1det(nn.Module):
    def __init__(self):
        super(AutoEncoder_1det, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(101, 20),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(20, 101),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(202, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 202),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
class WSClassifier(nn.Module):
    def __init__(self):
        super(WSClassifier, self).__init__()
        self.fc1 = nn.Linear(101, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 8)
        self.norm2 = nn.BatchNorm1d(8)
        self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        x = self.norm2(self.relu(self.fc2(x)))
        return self.fc4(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
class WSClassifier(nn.Module):
    def __init__(self):
        super(WSClassifier, self).__init__()
        self.fc1 = nn.Linear(101, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
class WSClassifier(nn.Module):
    def __init__(self):
        super(WSClassifier, self).__init__()
        self.fc1 = nn.Linear(202, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
ae = AutoEncoder().cuda()
print(sum(p.numel() for p in ae.parameters() if p.requires_grad))

In [ ]:
wsc = WSClassifier().cuda()
print(sum(p.numel() for p in wsc.parameters() if p.requires_grad))

In [ ]:
wsc

## data loading

In [ ]:
list_datatype = ["noise", "bbh", "sg", "glitch"]

In [ ]:
N_wsl_total = 3000;
N_wsl = {}
N_wsl["noise"] = int(0.75*N_wsl_total)
N_wsl["bbh"] = int(0.1*N_wsl_total)
N_wsl["sg"] = int(0.1*N_wsl_total)
N_wsl["glitch"] = int(0.05*N_wsl_total)

In [ ]:
N_wsl_total = 30000;
N_wsl = {}
N_wsl["noise"] = int(0.75*N_wsl_total)
N_wsl["bbh"] = int(0.1*N_wsl_total)
N_wsl["sg"] = int(0.1*N_wsl_total)
N_wsl["glitch"] = int(0.05*N_wsl_total)

In [ ]:
N_wsl_total = 30000;
N_wsl = {}
N_wsl["noise"] = int(0.85*N_wsl_total)
N_wsl["bbh"] = int(0.04*N_wsl_total)
N_wsl["sg"] = int(0.04*N_wsl_total)
N_wsl["glitch"] = int(0.07*N_wsl_total)

In [ ]:
renorm_factor_0 = 20;
renorm_factor_1 = 20;

realbkg = np.load('E://GWNMMAD_data/Tw_dataset/Datasets/background.npz')['data'].reshape(-1,200) / renorm_factor_0;
realbbh = np.load('E://GWNMMAD_data/Tw_dataset/Datasets/bbh_for_challenge.npy').reshape(-1,200) / renorm_factor_0;

realsg = np.load('E://GWNMMAD_data/Tw_dataset/Datasets/sglf_for_challenge.npy').reshape(-1,200) / renorm_factor_0;
# realglitch = np.load("../data/real_glitches_9998_4000Hz_25ms.npz")["strain_time_data"]
realglitch_L = np.load("../Data_cached/real_glitches_snrlt5_60132_4000Hz_25ms.npz")["strain_time_data"][:30000].reshape(-1,1,200) / renorm_factor_1
realglitch_H = np.load('../Data_cached/real_glitches_H_snrlt5_59732_4000Hz_25ms.npz')["strain_time_data"][:30000].reshape(-1,1,200) / renorm_factor_1

In [ ]:
## If one is using Chia-Jui's data

renorm_factor_0 = 20;
renorm_factor_1 = 20;

realbkg_L = np.load('../Data_cached/real_bkg_2202000_63917s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;
realbkg_H = np.load('../Data_cached/real_bkg_H_1466640_58803s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;

realbkg = np.concatenate((realbkg_L, realbkg_H), axis = 1).reshape(-1,200)

realbbh = np.load('../Data_cached/injected_BBH_1823_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;

realsg = np.load('../Data_cached/injected_lfsg_1835_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;
# realglitch = np.load("../data/real_glitches_9998_4000Hz_25ms.npz")["strain_time_data"]
realglitch_L = np.load("../Data_cached/real_glitches_snrlt5_60132_4000Hz_25ms.npz")["strain_time_data"][:30000].reshape(-1,1,200) / renorm_factor_1
realglitch_H = np.load('../Data_cached/real_glitches_H_snrlt5_59732_4000Hz_25ms.npz')["strain_time_data"][:30000].reshape(-1,1,200) / renorm_factor_1

Normal procedure for one glitch + one noise

In [ ]:
noise_first50k = realbkg.reshape(-1,2,200)[:30000]

glitch_L_noise_H = np.concatenate((realglitch_L, noise_first50k[:,[1],:]), axis = 1)
glitch_H_noise_L = np.concatenate((realglitch_H, noise_first50k[:,[0],:]), axis = 1)
realglitch = np.vstack((glitch_L_noise_H, glitch_H_noise_L))
np.random.shuffle(realglitch)
realglitch = realglitch.reshape(-1,200)
realbkg = realbkg[60000:]

In [ ]:
realbkg_reserved = realbkg[-int(0.2 * len(realbkg)):]
realbbh_reserved = realbbh[-int(0.2 * len(realbbh)):]
realsg_reserved = realsg[-int(0.2 * len(realsg)):]
realglitch_reserved = realglitch[-int(0.2 * len(realglitch)):]

realbkg = realbkg[:-int(0.2 * len(realbkg))]
realbbh = realbbh[:-int(0.2 * len(realbbh))-int(0.2 * len(realbbh))%2]
realsg = realsg[:-int(0.2 * len(realsg))-int(0.2 * len(realsg))%2]
realglitch = realglitch[:-int(0.2 * len(realglitch))]

In [ ]:
bkg_fft = abs(np.fft.rfft(realbkg))
bkg_fft = bkg_fft/np.linalg.norm([bkg_fft], axis=2).T

bbh_fft = abs(np.fft.rfft(realbbh))
bbh_fft = bbh_fft/np.linalg.norm([bbh_fft], axis=2).T

sg_fft = abs(np.fft.rfft(realsg))
sg_fft = sg_fft/np.linalg.norm([sg_fft], axis=2).T

glitch_fft = abs(np.fft.rfft(realglitch))
glitch_fft = glitch_fft/np.linalg.norm([glitch_fft], axis=2).T

Novel procedure. For training sample, the model is purely glitch trained. For testing and WSL sample, the model is one glitch + one noise

In [ ]:
num_glitch_for_AE = int(len(realglitch_L) * 0.8) - N_wsl['glitch'] // 2

In [ ]:
num_one_glitch_one_noise = int(len(realglitch_L) * 0.2) + N_wsl['glitch'] // 2

In [ ]:
np.random.shuffle(realglitch_L)
np.random.shuffle(realglitch_H)

In [ ]:
realglitch_L.shape

In [ ]:
realbkg.shape

In [ ]:
glitch_for_ae = np.concatenate((realglitch_L[:num_glitch_for_AE], realglitch_H[:num_glitch_for_AE]), axis=1).reshape(-1,200)
glitch_for_ae_fft = abs(np.fft.rfft(glitch_for_ae))
glitch_for_ae_fft = glitch_for_ae_fft/np.linalg.norm([glitch_for_ae_fft], axis=2).T

In [ ]:
noise_for_glitch_building = realbkg[:2 * num_one_glitch_one_noise].reshape(-1,2,200)

glitch_L_noise_H = np.concatenate((realglitch_L[-num_one_glitch_one_noise:], noise_for_glitch_building[:,[1],:]), axis = 1)
noise_L_glitch_H = np.concatenate((noise_for_glitch_building[:,[0],:], realglitch_H[-num_one_glitch_one_noise:]), axis = 1)
one_glitch_one_noise = np.vstack((glitch_L_noise_H, noise_L_glitch_H))
np.random.shuffle(one_glitch_one_noise)
one_glitch_one_noise = one_glitch_one_noise.reshape(-1,200)

realbkg = realbkg[2 * num_one_glitch_one_noise:]

In [ ]:
one_glitch_one_noise_fft = abs(np.fft.rfft(one_glitch_one_noise))
one_glitch_one_noise_fft = one_glitch_one_noise_fft/np.linalg.norm([one_glitch_one_noise_fft], axis=2).T

one_glitch_one_noise = one_glitch_one_noise / np.linalg.norm([one_glitch_one_noise_fft], axis=2).T

In [ ]:
realbkg_reserved = realbkg[-int(0.2 * len(realbkg)):]
realbbh_reserved = realbbh[-int(0.2 * len(realbbh)):]
realsg_reserved = realsg[-int(0.2 * len(realsg)):]
# realglitch_reserved = realglitch[-int(0.2 * len(realglitch)):]

realbkg = realbkg[:-int(0.2 * len(realbkg))]
realbbh = realbbh[:-int(0.2 * len(realbbh))-int(0.2 * len(realbbh))%2]
realsg = realsg[:-int(0.2 * len(realsg))-int(0.2 * len(realsg))%2]
# realglitch = realglitch[:-int(0.2 * len(realglitch))]
bkg_fft = abs(np.fft.rfft(realbkg))
bkg_fft = bkg_fft/np.linalg.norm([bkg_fft], axis=2).T

bbh_fft = abs(np.fft.rfft(realbbh))
bbh_fft = bbh_fft/np.linalg.norm([bbh_fft], axis=2).T

sg_fft = abs(np.fft.rfft(realsg))
sg_fft = sg_fft/np.linalg.norm([sg_fft], axis=2).T

# glitch_fft = abs(np.fft.rfft(realglitch))
# glitch_fft = glitch_fft/np.linalg.norm([glitch_fft], axis=2).T

In [ ]:
bkg_fft.shape

In [ ]:
glitch_fft.shape

In [ ]:
bbh_fft.shape

In [ ]:
bbh_fft.shape

In [ ]:
bkg_fft = bkg_fft.reshape(-1,202)

bbh_fft = bbh_fft.reshape(-1,202)

sg_fft = sg_fft.reshape(-1,202)

# glitch_fft = glitch_fft.reshape(-1,202)

glitch_for_ae_fft = glitch_for_ae_fft.reshape(-1,202)

one_glitch_one_noise_fft = one_glitch_one_noise_fft.reshape(-1,202)

In [ ]:
del realbkg_L, realbkg_H, realglitch_L, realglitch_H, glitch_L_noise_H, glitch_H_noise_L

In [ ]:
# dataset_raw = {};
# dataset_raw["noise"] = np.load("E://GWNMMAD_data/Tw_dataset/Datasets/background.npz")['data'].reshape(-1,200);
# dataset_raw["bbh"] = np.load("E://GWNMMAD_data/Tw_dataset/Datasets/bbh_for_challenge.npy").reshape(-1,200);
# dataset_raw["sg"] = np.load("E://GWNMMAD_data/Tw_dataset/Datasets/sglf_for_challenge.npy").reshape(-1,200);
# # realglitch = np.load("../data/real_glitches_9998_4000Hz_25ms.npz")["strain_time_data"]
# dataset_raw["glitch"] = np.load("../Data_cached/real_glitches_snrlt5_60132_4000Hz_25ms.npz")["strain_time_data"]

# dataset_wsl = {};
# dataset_ae = {};
# dataset_wsl_fft = {};
# dataset_ae_fft = {};

# for dt in list_datatype:
#     perm = np.random.permutation(len(dataset_raw[dt]))
#     nwsl = N_wsl[dt]
#     dataset_wsl[dt] = dataset_raw[dt][perm[:nwsl]]
#     dataset_wsl[dt] = dataset_wsl[dt] / np.linalg.norm([dataset_wsl[dt]], axis=2).T
#     dataset_wsl_fft[dt] = abs(np.fft.rfft(dataset_wsl[dt]))
#     dataset_wsl_fft[dt] = dataset_wsl_fft[dt]/np.linalg.norm([dataset_wsl_fft[dt]], axis=2).T
    
#     dataset_ae[dt]  = dataset_raw[dt][perm[nwsl:]]
#     dataset_ae[dt] = dataset_ae[dt] / np.linalg.norm([dataset_ae[dt]], axis=2).T
#     dataset_ae_fft[dt] = abs(np.fft.rfft(dataset_ae[dt]))
#     dataset_ae_fft[dt] = dataset_ae_fft[dt]/np.linalg.norm([dataset_ae_fft[dt]], axis=2).T
    
#     np.savetxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det.dat", perm)

In [ ]:
dataset_raw_fft = {}

dataset_raw_fft["noise"] = bkg_fft
dataset_raw_fft["bbh"] = bbh_fft
dataset_raw_fft["sg"] = sg_fft
dataset_raw_fft["glitch"] = glitch_fft

dataset_wsl = {};
dataset_ae = {};
dataset_wsl_fft = {};
dataset_ae_fft = {};

for dt in list_datatype:
    # perm = np.random.permutation(len(dataset_raw_fft[dt]))
    perm = np.loadtxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_v7_GWAK.dat").astype(int)
    nwsl = N_wsl[dt]
    dataset_wsl_fft[dt] = dataset_raw_fft[dt][perm[:nwsl]]
    # dataset_wsl[dt] = dataset_wsl[dt] / np.linalg.norm([dataset_wsl[dt]], axis=2).T
    # dataset_wsl_fft[dt] = abs(np.fft.rfft(dataset_wsl[dt]))
    # dataset_wsl_fft[dt] = dataset_wsl_fft[dt]/np.linalg.norm([dataset_wsl_fft[dt]], axis=2).T
    
    dataset_ae_fft[dt]  = dataset_raw_fft[dt][perm[nwsl:]]
    # dataset_ae[dt] = dataset_ae[dt] / np.linalg.norm([dataset_ae[dt]], axis=2).T
    # dataset_ae_fft[dt] = abs(np.fft.rfft(dataset_ae[dt]))
    # dataset_ae_fft[dt] = dataset_ae_fft[dt]/np.linalg.norm([dataset_ae_fft[dt]], axis=2).T
    
    # np.savetxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_"+version+"_2.dat", perm)

In [ ]:
list_datatype = ["noise", "bbh", "sg", "glitch"]

In [ ]:
dataset_raw_fft = {}

dataset_raw_fft["noise"] = bkg_fft
dataset_raw_fft["bbh"] = bbh_fft
dataset_raw_fft["sg"] = sg_fft
# dataset_raw_fft["glitch"] = glitch_fft

# dataset_wsl = {};
# dataset_ae = {};
dataset_wsl_fft = {};
dataset_ae_fft = {};

for dt in list_datatype[0:3]:
    perm = np.random.permutation(len(dataset_raw_fft[dt]))
    # perm = np.loadtxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_v7_GWAK.dat").astype(int)
    nwsl = N_wsl[dt]
    dataset_wsl_fft[dt] = dataset_raw_fft[dt][perm[:nwsl]]
    # dataset_wsl[dt] = dataset_wsl[dt] / np.linalg.norm([dataset_wsl[dt]], axis=2).T
    # dataset_wsl_fft[dt] = abs(np.fft.rfft(dataset_wsl[dt]))
    # dataset_wsl_fft[dt] = dataset_wsl_fft[dt]/np.linalg.norm([dataset_wsl_fft[dt]], axis=2).T
    
    dataset_ae_fft[dt]  = dataset_raw_fft[dt][perm[nwsl:]]
    # dataset_ae[dt] = dataset_ae[dt] / np.linalg.norm([dataset_ae[dt]], axis=2).T
    # dataset_ae_fft[dt] = abs(np.fft.rfft(dataset_ae[dt]))
    # dataset_ae_fft[dt] = dataset_ae_fft[dt]/np.linalg.norm([dataset_ae_fft[dt]], axis=2).T
    
    # np.savetxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_"+version+"_2.dat", perm)

In [ ]:
dataset_wsl_fft['glitch'] = one_glitch_one_noise_fft[:N_wsl['glitch']]

dataset_ae_fft['glitch'] = glitch_for_ae_fft

In [ ]:
np.linalg.norm(dataset_ae_fft['glitch'][0])

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_wsl_fft.keys():
    print(key)
    print(dataset_wsl_fft[key].shape)

In [ ]:
realglitch_reserved = one_glitch_one_noise_fft[N_wsl['glitch']:]

In [ ]:
bkg_reserved_fft = abs(np.fft.rfft(realbkg_reserved))
bkg_reserved_fft = bkg_reserved_fft/np.linalg.norm([bkg_reserved_fft], axis=2).T

bbh_reserved_fft = abs(np.fft.rfft(realbbh_reserved))
bbh_reserved_fft = bbh_reserved_fft/np.linalg.norm([bbh_reserved_fft], axis=2).T

sg_reserved_fft = abs(np.fft.rfft(realsg_reserved))
sg_reserved_fft = sg_reserved_fft/np.linalg.norm([sg_reserved_fft], axis=2).T

glitch_reserved_fft = abs(np.fft.rfft(realglitch_reserved))
glitch_reserved_fft = glitch_reserved_fft/np.linalg.norm([glitch_reserved_fft], axis=2).T

In [ ]:
bkg_reserved_fft = bkg_reserved_fft.reshape(-1,202)

bbh_reserved_fft = bbh_reserved_fft.reshape(-1,202)

sg_reserved_fft = sg_reserved_fft.reshape(-1,202)

glitch_reserved_fft = glitch_reserved_fft.reshape(-1,202)

In [ ]:
N_reserved_test = {}

N_reserved_test_total = 20000

N_reserved_test['glitch'] = int(0.05 * N_reserved_test_total)
N_reserved_test['noise'] = int(0.65 * N_reserved_test_total)
N_reserved_test['bbh'] = int(0.15 * N_reserved_test_total)
N_reserved_test['sg'] = int(0.15 * N_reserved_test_total)

dataset_reserved_test_collected = np.vstack((glitch_reserved_fft[:N_reserved_test['glitch']], bkg_reserved_fft[:N_reserved_test['noise']],
                                             bbh_reserved_fft[:N_reserved_test['bbh']], sg_reserved_fft[:N_reserved_test['sg']]))

In [ ]:
dataset_reserved_test_collected.shape

In [ ]:
dataset_wsl_fft_collected = np.empty((0, dataset_wsl_fft["glitch"].shape[1]))
for dt in sequence:
    dataset_wsl_fft_collected = np.vstack((dataset_wsl_fft_collected, dataset_wsl_fft[dt]))

In [ ]:
testset = dataset_wsl_fft_collected

## training

In [ ]:
version = "v9";

In [ ]:
epochs = 50;

In [ ]:
sequence = ["glitch", "noise", "bbh", "sg"];
ind2datatype = {};
datatype2ind = {};
for i, dt in enumerate(sequence):
    ind2datatype[i] = dt;
    datatype2ind[dt] = i;
    
torch.save(ind2datatype, "../Data_cached/SequentialTraining/WSL/sequence_"+version+".json")

In [ ]:
sequence = ["glitch", "noise", "bbh", "sg"];
dataset_wsl_fft_collected = np.empty((0, dataset_wsl_fft["glitch"].shape[1]))
for dt in sequence:
    dataset_wsl_fft_collected = np.vstack((dataset_wsl_fft_collected, dataset_wsl_fft[dt]))

In [ ]:
# N_bkg = len(bkg_reserved_fft);
# N_glitch = int(N_bkg/15);
# N_bbh = int(N_bkg*2/15);
# N_sg = int(N_bkg*2/15);

# testset = np.empty((N_bkg+N_glitch+N_bbh+N_sg, len(bkg_reserved_fft[0])))

# s = 0;
# testset[s : s+N_glitch] = glitch_reserved_fft[np.random.permutation(len(glitch_reserved_fft))[:N_glitch]];
# s += N_glitch;

# testset[s : s+N_bkg] = bkg_reserved_fft[np.random.permutation(len(bkg_reserved_fft))[:N_bkg]];
# s += N_bkg;

# testset[s : s+N_bbh] = bbh_reserved_fft[np.random.permutation(len(bbh_reserved_fft))[:N_bbh]];
# s += N_bbh;

# testset[s : s+N_sg] = sg_reserved_fft[np.random.permutation(len(sg_reserved_fft))[:N_sg]];
# s += N_sg;

# correct_ans = np.concatenate(([0]*N_glitch, [1]*N_bkg, [2]*N_bbh, [3]*N_sg))

# Nsample = {};
# Nsample["glitch"] = N_glitch;
# Nsample["noise"] = N_bkg;
# Nsample["bbh"] = N_bbh;
# Nsample["sg"] = N_sg

In [ ]:
# Ncut = 5;
# cutList = {};

# max_glitch = 0.0026;
# min_glitch = 0.001;
# cutList["glitch"] = np.linspace(min_glitch, max_glitch, Ncut);

# max_bkg = 0.0026;
# min_bkg = 0.001;
# cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);

# max_bbh = 0.0024;
# min_bbh = 0.0008;
# cutList["bbh"] = np.linspace(min_bbh, max_bbh, Ncut);

# max_sg = 0.003;
# min_sg = 0.0003;
# cutList["sg"] = np.linspace(min_sg, max_sg, Ncut);

# torch.save(cutList, "../Data_cached/SequentialTraining/WSL/cut_"+version+".json")

In [ ]:
Ncut = 5;
cutList = {};

max_glitch = 0.001;
min_glitch = 0.0024;
cutList["glitch"] = np.linspace(min_glitch, max_glitch, Ncut);

max_bkg = 0.0008;
min_bkg = 0.0018;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);

max_bbh = 0.0014;
min_bbh = 0.0024;
cutList["bbh"] = np.linspace(min_bbh, max_bbh, Ncut);

max_sg = 0.0008;
min_sg = 0.0032;
cutList["sg"] = np.linspace(min_sg, max_sg, Ncut);

torch.save(cutList, "../Data_cached/SequentialTraining/WSL/cut_"+version+".json")

In [ ]:
Ncut = 5;
cutList = {};

max_glitch = 0.001;
min_glitch = 0.0024;
cutList["glitch"] = np.linspace(min_glitch, max_glitch, Ncut);

max_bkg = 0.0008;
min_bkg = 0.0018;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);

max_bbh = 0.0014;
min_bbh = 0.0024;
cutList["bbh"] = np.linspace(min_bbh, max_bbh, Ncut);

max_sg = 0.0008;
min_sg = 0.0032;
cutList["sg"] = np.linspace(min_sg, max_sg, Ncut);

# torch.save(cutList, "../Data_cached/SequentialTraining/WSL/cut_"+version+".json")

In [ ]:
Ncut = 5;
cutList = {};

max_glitch_L = 0.0008;
min_glitch_L = 0.0024;
cutList["glitch_L"] = np.linspace(min_glitch_L, max_glitch_L, Ncut);

max_glitch_H = 0.0004;
min_glitch_H = 0.0024;
cutList["glitch_H"] = np.linspace(min_glitch_H, max_glitch_H, Ncut);

max_bkg = 0.0008;
min_bkg = 0.0018;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);

max_bbh = 0.0014;
min_bbh = 0.0024;
cutList["bbh"] = np.linspace(min_bbh, max_bbh, Ncut);

max_sg = 0.0008;
min_sg = 0.0032;
cutList["sg"] = np.linspace(min_sg, max_sg, Ncut);

torch.save(cutList, "../Data_cached/SequentialTraining/WSL/cut_"+version+".json")

In [ ]:
# models = {};
# models["glitch"] = torch.load("../Model_cached/4ae_3.pt")
# models["glitch"].cpu().eval()

In [ ]:
models = {};
models["glitch"] = torch.load('../Model_cached/2_det_oneglitchonenoise_Chia-Jui_glitch_trained_202-20-202.pt')
models["glitch"].cpu().eval()

In [ ]:
def trainAE(dataset, cutID, version, datatype):
    
    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainAE(dataset, cutID, version, datatype):
    
    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainWSC(dataset0, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE
    
    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

In [ ]:
model1 = trainWSC(dataset0, dataset1, "test").cpu().eval();

In [ ]:
import itertools

In [ ]:
cnt = 0;

ic = np.zeros(4, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
ic[2] = Ncut-1;
ic[3] = Ncut-1;

In [ ]:
cnt = 0;

ic = np.zeros(4, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
ic[2] = Ncut-1;
ic[3] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if cnt < 11:
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
        
    for iPrev in range(2):
        previousStep = ind2datatype[iPrev];
        modelPrev = models[previousStep]; # previous step AE
        
        # train the WSC according to previous AE's cut
        
        dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
        dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
        dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
        dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
        dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
        model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
        models[previousStep+"_WSC"] = model;
        
        # filter the data according to previous WSC
        for j in range(iPrev, 4):
            dt = ind2datatype[j];
            dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
            data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
        # train the current step AE
        currentStep = ind2datatype[iPrev+1];
        model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
        models[currentStep] = model;
        
    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    
#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
testset = dataset_wsl_fft_collected

In [ ]:
testset = dataset_reserved_test_collected

In [ ]:
testset.shape

In [ ]:
correct_ans = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [2]*N_wsl['bbh'], [3]*N_wsl['sg']))

In [ ]:
correct_ans_withoutsignal = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [-1]*(N_wsl['bbh']+N_wsl['sg'])))

In [ ]:
correct_ans_withoutsignal = np.hstack(([0]*N_reserved_test['glitch'], [1]*N_reserved_test['noise'], [2]*N_reserved_test['bbh'], [3]*N_reserved_test['sg']))

In [ ]:
correct_ans.shape

In [ ]:
correct_ans_withoutsignal.shape

In [ ]:
ic[3] = 0

In [ ]:
list_datatype_withoutsignal = ["noise", "glitch"]

In [ ]:
ic_withoutsignal = np.array([0,0])

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    for datatype in list_datatype_withoutsignal:
        if datatype == 'sg':
            dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
            err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
        # elif datatype == 'noise':
        #     dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
        #     err[datatype] = np.var(testset-dcd[datatype], axis=1)
            
        else:
            dcd[datatype] = nn.Sigmoid()(models[datatype + "_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
        
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5)
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    datatype = 'noise'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
listResult['FPR']

In [ ]:
# This is the result for 0.05, 0.65, 0.15, 0.15

print(listResult['cut'][np.argmin(listResult['FPR'])])
print(listResult['FPR'].min())

In [ ]:
# This is the result for 0.1, 0.3, 0.3, 0.3

print(listResult['cut'][np.argmin(listResult['FPR'])])
print(listResult['FPR'].min())

In [ ]:
np.argwhere(ans[-6000:] == 1).shape

In [ ]:
N_wsl['noise']

In [ ]:
np.argwhere(ans[1500:-6000] == 1).shape

In [ ]:
np.argwhere(ans[1500:-6000] == -1).shape

In [ ]:
np.argwhere(ans[-6000:] == -1).shape

In [ ]:
np.argwhere(ans[1500:-6000] == -1).shape

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "04" + "_"+version+".json")

In [ ]:
dataset_wsl_fft

In [ ]:
model = models['noise']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model

In [ ]:
model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
listResult['FPR']

In [ ]:
listResult['FPR'].min()

In [ ]:
listResult['FPR'][np.argwhere(listResult['cut'] == [2,0]).flatten()]

In [ ]:
listResult['FPR']

In [ ]:
135 / 22500

In [ ]:
cutList

In [ ]:
foo = np.array([ [0.01*A, np.sum(np.all(listResult["accuracy_4"] >= 0.01*A, axis=1))/len(listResult["accuracy_4"])] for A in range(101)])
plt.plot(foo[:, 0], foo[:, 1])
plt.title('4-class accuracy')
plt.xlabel('minimal accuracy larger than...')
plt.ylabel('percentage')

In [ ]:
listResult["accuracy_4"][np.argwhere(np.all(listResult["accuracy_4"]>0.8, axis=1)).flatten()]

In [ ]:
foo = np.array([ [0.01*A, np.sum(np.all(listResult["accuracy_2"] >= 0.01*A, axis=1))/len(listResult["accuracy_2"])] for A in range(101)])
plt.plot(foo[:, 0], foo[:, 1])
plt.title('2-class accuracy')
plt.xlabel('minimal accuracy larger than...')
plt.ylabel('percentage')

In [ ]:
listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset)

In [ ]:
listResult['total_accuracy'].max()

In [ ]:
listResult["accuracy_4"][np.argmax(listResult["accuracy_4"]>0.8).flatten()]

In [ ]:
np.argwhere(np.all(np.array(list(itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)))) == np.array([3,1,3]), axis = 1))

In [ ]:
np.array(list(itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut))))

In [ ]:
ic

In [ ]:
cutList

# Result for Chia-Jui noise trained model

In [ ]:
testset = dataset_wsl_fft_collected

In [ ]:
testset = dataset_reserved_test_collected

In [ ]:
testset.shape

In [ ]:
correct_ans = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [2]*N_wsl['bbh'], [3]*N_wsl['sg']))

In [ ]:
correct_ans_withoutsignal = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [-1]*(N_wsl['bbh']+N_wsl['sg'])))

In [ ]:
correct_ans_withoutsignal = np.hstack(([0]*N_reserved_test['glitch'], [1]*N_reserved_test['noise'], [2]*N_reserved_test['bbh'], [3]*N_reserved_test['sg']))

In [ ]:
correct_ans.shape

In [ ]:
correct_ans_withoutsignal.shape

In [ ]:
ic[3] = 0

In [ ]:
list_datatype_withoutsignal = ["noise", "glitch"]

In [ ]:
ic_withoutsignal = np.array([0,0])

In [ ]:
Ncut = 5

In [ ]:
version = 'v7'

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    for datatype in list_datatype_withoutsignal:
        if datatype == 'sg':
            dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
            err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
        # elif datatype == 'noise':
        #     dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
        #     err[datatype] = np.var(testset-dcd[datatype], axis=1)
            
        else:
            dcd[datatype] = nn.Sigmoid()(models[datatype + "_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
        
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5)
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    datatype = 'noise'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    for datatype in list_datatype_withoutsignal:
        if datatype == 'sg':
            dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
            err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
        # elif datatype == 'noise':
        #     dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
        #     err[datatype] = np.var(testset-dcd[datatype], axis=1)
            
        else:
            dcd[datatype] = nn.Sigmoid()(models[datatype + "_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
        
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5)
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    datatype = 'noise'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
listResult['FPR']

In [ ]:
listResult['FPR']

In [ ]:
listResult['FPR']

In [ ]:
print(listResult['cut'][np.argmin(listResult['FPR'])])
print(listResult['FPR'].min())

In [ ]:
# This is the result for 0.05, 0.65, 0.15, 0.15

print(listResult['cut'][np.argmin(listResult['FPR'])])
print(listResult['FPR'].min())

In [ ]:
# This is the result for 0.1, 0.3, 0.3, 0.3

print(listResult['cut'][np.argmin(listResult['FPR'])])
print(listResult['FPR'].min())

In [ ]:
np.argwhere(ans[-6000:] == 1).shape

In [ ]:
N_wsl['noise']

In [ ]:
np.argwhere(ans[1500:-6000] == 1).shape

In [ ]:
np.argwhere(ans[1500:-6000] == -1).shape

In [ ]:
np.argwhere(ans[-6000:] == -1).shape

In [ ]:
np.argwhere(ans[1500:-6000] == -1).shape

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "00" + "_"+version+".json")

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "00" + "_v6.json")

In [ ]:
dataset_wsl_fft

In [ ]:
model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(0.0024, color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
# GWAK data + Chia-Jui model

model = models['noise']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
# plt.axvline(0.0018, color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + GWAK model

model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + GWAK model

model = models['noise']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.axvline(0.0024, color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model

In [ ]:
model = models['glitch']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model = models['noise']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.axvline(0.0018, color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model = models['noise']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = model(torch.FloatTensor(bkg_filtered))[1].detach().numpy()
dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(np.var(bbh_filtered-dcd_bbh, axis=1))[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
plt.ylim(0,2000)

In [ ]:
model

In [ ]:
# GWAK data + Chia-Jui model

model = models['glitch_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.axvline(0.5, color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
# GWAK data + Chia-Jui model

model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
# foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + GWAK model

model = models['glitch_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with glitch")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + GWAK model

model = models['glitch_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + GWAK model

model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + Chia-Jui model

model = models['glitch_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
# plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.axvline(0.5, color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
# Chia-Jui data + Chia-Jui model

model = models['glitch_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']


# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_fft))).detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

In [ ]:
listResult['FPR']

In [ ]:
listResult['FPR'].min()

In [ ]:
listResult['FPR'][np.argwhere(listResult['cut'] == [2,0]).flatten()]

In [ ]:
listResult['FPR']

In [ ]:
135 / 22500

In [ ]:
cutList

In [ ]:
foo = np.array([ [0.01*A, np.sum(np.all(listResult["accuracy_4"] >= 0.01*A, axis=1))/len(listResult["accuracy_4"])] for A in range(101)])
plt.plot(foo[:, 0], foo[:, 1])
plt.title('4-class accuracy')
plt.xlabel('minimal accuracy larger than...')
plt.ylabel('percentage')

In [ ]:
listResult["accuracy_4"][np.argwhere(np.all(listResult["accuracy_4"]>0.8, axis=1)).flatten()]

In [ ]:
foo = np.array([ [0.01*A, np.sum(np.all(listResult["accuracy_2"] >= 0.01*A, axis=1))/len(listResult["accuracy_2"])] for A in range(101)])
plt.plot(foo[:, 0], foo[:, 1])
plt.title('2-class accuracy')
plt.xlabel('minimal accuracy larger than...')
plt.ylabel('percentage')

In [ ]:
listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset)

In [ ]:
listResult['total_accuracy'].max()

In [ ]:
listResult["accuracy_4"][np.argmax(listResult["accuracy_4"]>0.8).flatten()]

In [ ]:
np.argwhere(np.all(np.array(list(itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)))) == np.array([3,1,3]), axis = 1))

In [ ]:
np.array(list(itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut))))

In [ ]:
ic

In [ ]:
cutList

# Let's think about using 2 det for the glitch

In [ ]:
models = {};
models["glitch_L"] = torch.load('../Model_cached/1_det_L_glitch_trained_with_permutation_1.pt')
models["glitch_H"] = torch.load('../Model_cached/1_det_H_glitch_trained_with_permutation_1.pt')
models["glitch_L"].cpu().eval()
models["glitch_H"].cpu().eval()

In [ ]:
models

In [ ]:
cutList

In [ ]:
import itertools

In [ ]:
cnt = 0;

ic = np.zeros(5, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
ic[2] = Ncut-1;
ic[3] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    # if cnt < 11:
    #     continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
    
    iPrev = 0
    
    previousStep = ind2datatype[iPrev];
    modelPrev_L = models['glitch_L']; # previous step AE
    modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_filtered[:,:101]))[1].detach().numpy();
    dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_filtered[:,101:]))[1].detach().numpy();
    dataset1 = dataset_wsl_filtered[np.logical_and(np.var(dataset_wsl_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
                                                   np.var(dataset_wsl_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[0]])]
    
    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;
    
    # filter the data according to previous WSC
    for j in range(iPrev, 4):
        dt = ind2datatype[j];
        dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][dcd>0.5]
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE
    currentStep = ind2datatype[iPrev+1];
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
       
     
    for iPrev in range(1,2):
        previousStep = ind2datatype[iPrev];
        modelPrev = models[previousStep]; # previous step AE
        
        # train the WSC according to previous AE's cut
        
        dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
        dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
        dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
        dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
        dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
        model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
        models[previousStep+"_WSC"] = model;
        
        # filter the data according to previous WSC
        for j in range(iPrev, 4):
            dt = ind2datatype[j];
            dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
            data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
        # train the current step AE
        currentStep = ind2datatype[iPrev+1];
        model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
        models[currentStep] = model;
        
    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    
#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
1

In [ ]:
cnt = 0;

ic = np.zeros(5, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
ic[3] = Ncut-1;
ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1], ic[2] in itertools.product(np.arange(Ncut), np.arange(Ncut),  np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    # if cnt < 11:
    #     continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
    
    iPrev = 0
    
    previousStep = ind2datatype[iPrev];
    modelPrev_L = models['glitch_L']; # previous step AE
    modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_filtered[:,:101]))[1].detach().numpy();
    dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_filtered[:,101:]))[1].detach().numpy();
    dataset1 = dataset_wsl_filtered[np.logical_and(np.var(dataset_wsl_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
                                                   np.var(dataset_wsl_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;
    
    # filter the data according to previous WSC
    for j in range(iPrev, 4):
        dt = ind2datatype[j];
        dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][dcd>0.5]
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE
    currentStep = ind2datatype[iPrev+1];
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
       
     
    for iPrev in range(1,2):
        previousStep = ind2datatype[iPrev];
        modelPrev = models[previousStep]; # previous step AE
        
        # train the WSC according to previous AE's cut
        
        dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
        dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
        dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev+1]]]
        
        dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
        dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev+1]]]
        
        model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
        models[previousStep+"_WSC"] = model;
        
        # filter the data according to previous WSC
        for j in range(iPrev, 4):
            dt = ind2datatype[j];
            dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
            data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
        # train the current step AE
        currentStep = ind2datatype[iPrev+1];
        model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
        models[currentStep] = model;
        
    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
    print(models.keys())
    
#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
dataset_wsl_filtered.shape

In [ ]:
dataset1.shape

In [ ]:
dataset0.shape

In [ ]:
dataset_ae_fft['glitch'].shape

In [ ]:
dataset_ae_fft['noise'].shape

In [ ]:
dataset0.shape

## Two detector for glitch, but WSL for each detector

### The WSL model is already pretrained

In [ ]:
class WSClassifier_Onedetector(nn.Module):
    def __init__(self):
        super(WSClassifier_Onedetector, self).__init__()
        self.fc1 = nn.Linear(101, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
dataset_ae_fft['bbh'].shape

In [ ]:
dataset_wsl_fft['bbh'].shape

In [ ]:
version = 'v10'

In [ ]:
cutList

In [ ]:
Ncut = 5;
cutList = {};

max_glitch_L = 0.0024;
min_glitch_L = 0.0008;
cutList["glitch_L"] = np.linspace(min_glitch_L, max_glitch_L, Ncut);

max_glitch_H = 0.0024;
min_glitch_H = 0.0004;
cutList["glitch_H"] = np.linspace(min_glitch_H, max_glitch_H, Ncut);

max_bkg = 0.0008;
min_bkg = 0.0018;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);

# max_bbh = 0.0014;
# min_bbh = 0.0024;
# cutList["bbh"] = np.linspace(min_bbh, max_bbh, Ncut);

# max_sg = 0.0008;
# min_sg = 0.0032;
# cutList["sg"] = np.linspace(min_sg, max_sg, Ncut);

# torch.save(cutList, "../Data_cached/SequentialTraining/WSL/cut_"+version+".json")

In [ ]:
models = {}

In [ ]:
version

In [ ]:
cnt = 0;

ic = np.zeros(5, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
ic[3] = Ncut-1;
ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1], ic[2] in itertools.product(np.arange(Ncut), np.arange(Ncut),  np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if cnt < 102:
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
    
    cutID = "".join(str(ic[j]) for j in range(3)) + "_"+version
    
    iPrev = 0
    
    previousStep = ind2datatype[iPrev];
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_filtered[np.logical_and(np.var(dataset_wsl_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    models[previousStep+"_WSC_L"] = model_L
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for j in range(iPrev, 4):
        dt = ind2datatype[j];
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE
    currentStep = ind2datatype[iPrev+1];
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
       
     
    for iPrev in range(1,2):
        previousStep = ind2datatype[iPrev];
        modelPrev = models[previousStep]; # previous step AE
        
        # train the WSC according to previous AE's cut
        
        dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
        dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
        dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev+1]]]
        
        dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
        dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev+1]]]
        
        model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
        models[previousStep+"_WSC"] = model;
        
        # filter the data according to previous WSC
        for j in range(iPrev, 4):
            dt = ind2datatype[j];
            dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
            data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
        # train the current step AE
        currentStep = ind2datatype[iPrev+1];
        model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
        models[currentStep] = model;
        
    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
    print(models.keys())
    
#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
data_filtered.keys()

## Evaluating the model

In [ ]:
import itertools

In [ ]:
list_datatype_withoutsignal = ["noise", "glitch"]

In [ ]:
sequence = ["glitch", "noise", "bbh", "sg"];
dataset_wsl_fft_collected = np.empty((0, dataset_wsl_fft["glitch"].shape[1]))
for dt in sequence:
    dataset_wsl_fft_collected = np.vstack((dataset_wsl_fft_collected, dataset_wsl_fft[dt]))

In [ ]:
testset = dataset_wsl_fft_collected

In [ ]:
ic_withoutsignal = np.zeros(3, dtype = int)

In [ ]:
list_datatype_withoutsignal

In [ ]:
correct_ans_withoutsignal = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [-1]*(N_wsl['bbh']+N_wsl['sg'])))

In [ ]:
correct_ans_withoutsignal.shape

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)+1), len(list_datatype_withoutsignal)+1), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)+1), len(testset)), dtype="int");
listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)+1), len(list_datatype_withoutsignal)))
listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)+1), 2))
cnt = 0

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)+1),1))


for ic_withoutsignal[0], ic_withoutsignal[1], ic_withoutsignal[2] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(3)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    dcd['noise'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] <= 0.5, dcd['glitch_H'] <= 0.5))
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    datatype = 'noise'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
listResult.keys()

In [ ]:
listResult['cut'][np.argwhere(listResult['FPR'] < 0.7)]

In [ ]:
np.argmax(listResult['FPR'])

In [ ]:
listResult['FPR']

In [ ]:
listResult['cut'][:,0]

In [ ]:
listResult['cut'][:,[0]] == [idx_1]

In [ ]:
idx_1 = 1
idx_2 = 0

np.argwhere(np.all(listResult['cut'][:,:2] == np.array([idx_1, idx_2]), axis = 1))

In [ ]:
idx_1 = 1
idx_2 = 3

np.argwhere(np.all(listResult['cut'][:,:2] == np.array([idx_1, idx_2]), axis = 1))

In [ ]:
models

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join([]) + "_"+version+".json")

In [ ]:
model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
bbh_filtered = dataset_wsl_fft['bbh']
sg_filtered = dataset_wsl_fft['sg']

# c0[currentStep] = 0.0022;

dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

# foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
# foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
plt.title("trained with noise")
# plt.axvline(c0[currentStep], color="k", linestyle="--")
plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
plt.legend()
# plt.ylim(0,2000)

## Including the CNN. 

### A quick check for roburtness

#### Data processing part

Setting hyperparameters

In [ ]:
N_wsl_total = 30000;
N_wsl = {}
N_wsl["noise"] = int(0.65*N_wsl_total)
N_wsl["bbh"] = int(0.1*N_wsl_total)
N_wsl["sglf"] = int(0.1*N_wsl_total)
N_wsl["sghf"] = int(0.1*N_wsl_total)
N_wsl["glitch"] = int(0.05*N_wsl_total)


snr_range = ['5-12','12-24','24-48','48-96']
ratio = [0.25, 0.25, 0.25, 0.25]
list_datatype = ["noise", "bbh", "sglf", "sghf", "glitch"]

In [ ]:
list_datatype_full = ['noise']

for snr in snr_range:
    list_datatype_full.append('bbh' + snr)
    list_datatype_full.append('sglf' + snr)
    list_datatype_full.append('sghf' + snr)
    
list_datatype_full.append('glitch')

In [ ]:
sequence = ['glitch','noise', 
            'bbh5-12', 'bbh12-24', 'bbh24-48', 'bbh48-96',
            'sglf5-12', 'sglf12-24', 'sglf24-48', 'sglf48-96',
            'sghf5-12', 'sghf12-24', 'sghf24-48', 'sghf48-96',]

In [ ]:
assert len(snr_range) == len(ratio)

for i in range(len(ratio)):

    N_wsl['bbh' + snr_range[i]] = int(ratio[i] * N_wsl['bbh'])
    
    N_wsl['sglf' + snr_range[i]] = int(ratio[i] * N_wsl['sglf'])
    
    N_wsl['sghf' + snr_range[i]] = int(ratio[i] * N_wsl['sghf'])

In [ ]:
### Introducing Chia-Jui's data

renorm_factor_0 = 20;
renorm_factor_1 = 20;

realbkg_L = np.load('../Data_cached/real_bkg_2202000_63917s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;
realbkg_H = np.load('../Data_cached/real_bkg_H_1466640_58803s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;

realbkg = np.concatenate((realbkg_L, realbkg_H), axis = 1).reshape(-1,200)

# realbbh = np.load('../Data_cached/injected_BBH_1823_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;

# realsg = np.load('../Data_cached/injected_lfsg_1835_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;
# realglitch = np.load("../data/real_glitches_9998_4000Hz_25ms.npz")["strain_time_data"]

realbbh_list = {}
realsglf_list = {}
realsghf_list = {}


for snr in snr_range:
    realbbh_list[snr] = np.load('../Data_cached/injected_BBH_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)
    realsglf_list[snr] = np.load('../Data_cached/injected_SGLF_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)
    realsghf_list[snr] = np.load('../Data_cached/injected_SGHF_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)

    

realglitch_L = np.load("../Data_cached/real_glitches_snrlt5_60132_4000Hz_25ms.npz")["strain_time_data"][:50000].reshape(-1,1,200) / renorm_factor_1
realglitch_H = np.load('../Data_cached/real_glitches_H_snrlt5_59732_4000Hz_25ms.npz')["strain_time_data"][:50000].reshape(-1,1,200) / renorm_factor_1

In [ ]:
realglitch_L = np.load("../Data_cached/real_glitches_snrlt5_60132_4000Hz_25ms.npz")["strain_time_data"][:30000].reshape(-1,1,200) / renorm_factor_1
realglitch_H = np.load('../Data_cached/real_glitches_H_snrlt5_59732_4000Hz_25ms.npz')["strain_time_data"][:30000].reshape(-1,1,200) / renorm_factor_1

Novel procedure. For training sample, the model is purely glitch trained. For testing and WSL sample, the model is one glitch + one noise

In [ ]:
num_glitch_for_AE = int(len(realglitch_L) * 0.8) - N_wsl['glitch'] // 2

In [ ]:
num_one_glitch_one_noise = int(len(realglitch_L) * 0.2) + N_wsl['glitch'] // 2

In [ ]:
np.random.shuffle(realglitch_L)
np.random.shuffle(realglitch_H)

In [ ]:
realglitch_L.shape

In [ ]:
realbkg.shape

In [ ]:
glitch_for_ae = np.concatenate((realglitch_L[:num_glitch_for_AE], realglitch_H[:num_glitch_for_AE]), axis=1).reshape(-1,200)
glitch_for_ae_fft = abs(np.fft.rfft(glitch_for_ae))
glitch_for_ae_fft = glitch_for_ae_fft/np.linalg.norm([glitch_for_ae_fft], axis=2).T

In [ ]:
noise_for_glitch_building = realbkg[:2 * num_one_glitch_one_noise].reshape(-1,2,200)

glitch_L_noise_H = np.concatenate((realglitch_L[-num_one_glitch_one_noise:], noise_for_glitch_building[:,[1],:]), axis = 1)
noise_L_glitch_H = np.concatenate((noise_for_glitch_building[:,[0],:], realglitch_H[-num_one_glitch_one_noise:]), axis = 1)
one_glitch_one_noise = np.vstack((glitch_L_noise_H, noise_L_glitch_H))
np.random.shuffle(one_glitch_one_noise)
one_glitch_one_noise = one_glitch_one_noise.reshape(-1,200)

realbkg = realbkg[2 * num_one_glitch_one_noise:]

In [ ]:
one_glitch_one_noise_fft = abs(np.fft.rfft(one_glitch_one_noise))
one_glitch_one_noise_fft = one_glitch_one_noise_fft / np.linalg.norm([one_glitch_one_noise_fft], axis=2).T

one_glitch_one_noise = one_glitch_one_noise / np.linalg.norm([one_glitch_one_noise], axis = 2).T

In [ ]:
realbkg_reserved = realbkg[-int(0.2 * len(realbkg)):]
# realbbh_reserved = realbbh[-int(0.2 * len(realbbh)):]
# realsg_reserved = realsg[-int(0.2 * len(realsg)):]
# realglitch_reserved = realglitch[-int(0.2 * len(realglitch)):]

realbkg = realbkg[:-int(0.2 * len(realbkg))]
# realbbh = realbbh[:-int(0.2 * len(realbbh))-int(0.2 * len(realbbh))%2]
# realsg = realsg[:-int(0.2 * len(realsg))-int(0.2 * len(realsg))%2]
# realglitch = realglitch[:-int(0.2 * len(realglitch))]

realbkg = realbkg / np.linalg.norm([realbkg], axis = 2).T

bkg_fft = abs(np.fft.rfft(realbkg))
bkg_fft = bkg_fft/np.linalg.norm([bkg_fft], axis=2).T

# bbh_fft = abs(np.fft.rfft(realbbh))
# bbh_fft = bbh_fft/np.linalg.norm([bbh_fft], axis=2).T

# sg_fft = abs(np.fft.rfft(realsg))
# sg_fft = sg_fft/np.linalg.norm([sg_fft], axis=2).T

# glitch_fft = abs(np.fft.rfft(realglitch))
# glitch_fft = glitch_fft/np.linalg.norm([glitch_fft], axis=2).T

In [ ]:
realbbh_reserved_list = {}
realsglf_reserved_list = {}
realsghf_reserved_list = {}

realbbh_fft_list = {}
realsglf_fft_list = {}
realsghf_fft_list = {}


for snr in snr_range:
    realbbh_reserved_list[snr] = realbbh_list[snr][-int(0.2 * len(realbbh_list[snr]))-int(0.2 * len(realbbh_list[snr]))%2:]
    realbbh_list[snr] = realbbh_list[snr][:-int(0.2 * len(realbbh_list[snr]))-int(0.2 * len(realbbh_list[snr]))%2]

    realbbh_fft_list[snr] = abs(np.fft.rfft(realbbh_list[snr]))
    realbbh_fft_list[snr] = realbbh_fft_list[snr]/np.linalg.norm([realbbh_fft_list[snr]], axis=2).T
    
    realbbh_list[snr] = realbbh_list[snr]/np.linalg.norm([realbbh_list[snr]], axis=2).T
    
    
    realsglf_reserved_list[snr] = realsglf_list[snr][-int(0.2 * len(realsglf_list[snr]))-int(0.2 * len(realsglf_list[snr]))%2:]
    realsglf_list[snr] = realsglf_list[snr][:-int(0.2 * len(realsglf_list[snr]))-int(0.2 * len(realsglf_list[snr]))%2]

    realsglf_fft_list[snr] = abs(np.fft.rfft(realsglf_list[snr]))
    realsglf_fft_list[snr] = realsglf_fft_list[snr]/np.linalg.norm([realsglf_fft_list[snr]], axis=2).T
    
    realsglf_list[snr] = realsglf_list[snr]/np.linalg.norm([realsglf_list[snr]], axis=2).T
    
    
    realsghf_reserved_list[snr] = realsghf_list[snr][-int(0.2 * len(realsghf_list[snr]))-int(0.2 * len(realsghf_list[snr]))%2:]
    realsghf_list[snr] = realsghf_list[snr][:-int(0.2 * len(realsghf_list[snr]))-int(0.2 * len(realsghf_list[snr]))%2]

    realsghf_fft_list[snr] = abs(np.fft.rfft(realsghf_list[snr]))
    realsghf_fft_list[snr] = realsghf_fft_list[snr]/np.linalg.norm([realsghf_fft_list[snr]], axis=2).T
    
    realsghf_list[snr] = realsghf_list[snr]/np.linalg.norm([realsghf_list[snr]], axis=2).T
    

In [ ]:
bkg_fft = bkg_fft.reshape(-1,202)

# bbh_fft = bbh_fft.reshape(-1,202)

# sg_fft = sg_fft.reshape(-1,202)

# glitch_fft = glitch_fft.reshape(-1,202)

glitch_for_ae_fft = glitch_for_ae_fft.reshape(-1,202)

one_glitch_one_noise_fft = one_glitch_one_noise_fft.reshape(-1,202)

In [ ]:
dataset_raw = {}
dataset_raw_fft = {}

dataset_raw["noise"] = realbkg.reshape(-1,400)
# dataset_raw_fft["bbh"] = bbh_fft
# dataset_raw_fft["sg"] = sg_fft
for snr in snr_range:
    dataset_raw['bbh' + snr] = realbbh_list[snr].reshape(-1,400)
    dataset_raw['sglf' + snr] = realsglf_list[snr].reshape(-1,400)
    dataset_raw['sghf' + snr] = realsghf_list[snr].reshape(-1,400)
# dataset_raw["glitch"] = glitch.reshape(-1,400)


dataset_raw_fft["noise"] = bkg_fft
# dataset_raw_fft["bbh"] = bbh_fft
# dataset_raw_fft["sg"] = sg_fft
for snr in snr_range:
    dataset_raw_fft['bbh' + snr] = realbbh_fft_list[snr].reshape(-1,202)
    dataset_raw_fft['sglf' + snr] = realsglf_fft_list[snr].reshape(-1,202)
    dataset_raw_fft['sghf' + snr] = realsghf_fft_list[snr].reshape(-1,202)
# dataset_raw_fft["glitch"] = glitch_fft

dataset_wsl = {};
dataset_ae = {};
dataset_wsl_fft = {};
dataset_ae_fft = {};

for dt in list_datatype_full[:-1]:
    perm = np.random.permutation(len(dataset_raw_fft[dt]))
    # perm = np.loadtxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_v7_GWAK.dat").astype(int)
    
    nwsl = N_wsl[dt]
    dataset_wsl[dt] = dataset_raw[dt][perm[:nwsl]]
    dataset_wsl_fft[dt] = dataset_raw_fft[dt][perm[:nwsl]]
    # dataset_wsl[dt] = dataset_wsl[dt] / np.linalg.norm([dataset_wsl[dt]], axis=2).T
    # dataset_wsl_fft[dt] = abs(np.fft.rfft(dataset_wsl[dt]))
    # dataset_wsl_fft[dt] = dataset_wsl_fft[dt]/np.linalg.norm([dataset_wsl_fft[dt]], axis=2).T
    
    dataset_ae[dt] = dataset_raw[dt][perm[nwsl:]]
    dataset_ae_fft[dt]  = dataset_raw_fft[dt][perm[nwsl:]]
    # dataset_ae[dt] = dataset_ae[dt] / np.linalg.norm([dataset_ae[dt]], axis=2).T
    # dataset_ae_fft[dt] = abs(np.fft.rfft(dataset_ae[dt]))
    # dataset_ae_fft[dt] = dataset_ae_fft[dt]/np.linalg.norm([dataset_ae_fft[dt]], axis=2).T
    
    # np.savetxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_"+version+"_2.dat", perm)
    
dataset_ae['bbh'] = np.zeros((0,400))
dataset_ae['sglf'] = np.zeros((0,400))
dataset_ae['sghf'] = np.zeros((0,400))

dataset_ae_fft['bbh'] = np.zeros((0,202))
dataset_ae_fft['sglf'] = np.zeros((0,202))
dataset_ae_fft['sghf'] = np.zeros((0,202))

for snr in snr_range:
    dataset_ae['bbh'] = np.append(dataset_ae['bbh'], dataset_ae['bbh'+snr])
    dataset_ae_fft['bbh'] = np.append(dataset_ae_fft['bbh'], dataset_ae_fft['bbh'+snr])
    
    dataset_ae['sglf'] = np.append(dataset_ae['sglf'], dataset_ae['sglf'+snr])
    dataset_ae_fft['sglf'] = np.append(dataset_ae_fft['sglf'], dataset_ae_fft['sglf'+snr])
    
    dataset_ae['sghf'] = np.append(dataset_ae['sghf'], dataset_ae['sghf'+snr])
    dataset_ae_fft['sghf'] = np.append(dataset_ae_fft['sghf'], dataset_ae_fft['sghf'+snr])

dataset_ae['bbh'] = dataset_ae['bbh'].reshape(-1,400)
dataset_ae['sglf'] = dataset_ae['sglf'].reshape(-1,400)
dataset_ae['sghf'] = dataset_ae['sghf'].reshape(-1,400)

dataset_ae_fft['bbh'] = dataset_ae_fft['bbh'].reshape(-1,202)
dataset_ae_fft['sglf'] = dataset_ae_fft['sglf'].reshape(-1,202)
dataset_ae_fft['sghf'] = dataset_ae_fft['sghf'].reshape(-1,202)

In [ ]:
dataset_raw[dt].shape

In [ ]:
dataset_wsl[dt]

In [ ]:
len(dataset_raw_fft[dt])

In [ ]:
dataset_wsl_fft['glitch'] = one_glitch_one_noise_fft[:N_wsl['glitch']]
dataset_wsl['glitch'] = one_glitch_one_noise[:2 * N_wsl['glitch']].reshape(-1,400)

dataset_ae_fft['glitch'] = glitch_for_ae_fft

# Missing dataset ae ['glitch'] here

In [ ]:
dataset_wsl.keys()

In [ ]:
for key in dataset_wsl.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
for key in dataset_ae.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
for key in dataset_wsl_fft.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
dataset_wsl_fft.keys()

In [ ]:
dataset_ae_fft

In [ ]:
for key in dataset_ae.keys():
    print(key)
    print(dataset_ae[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_wsl.keys():
    print(key)
    print(dataset_wsl[key].shape)

In [ ]:
for key in dataset_wsl_fft.keys():
    print(key)
    print(dataset_wsl_fft[key].shape)

In [ ]:
dataset_wsl_fft_collected = np.empty((0, 202))
for dt in sequence:
    dataset_wsl_fft_collected = np.vstack((dataset_wsl_fft_collected, dataset_wsl_fft[dt]))

In [ ]:
dataset_wsl_collected = np.empty((0, 400))
for dt in sequence:
    dataset_wsl_collected = np.vstack((dataset_wsl_collected, dataset_wsl[dt]))

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
dataset_wsl_collected.shape

#### Model definition

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(202, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 202),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
import torch.nn as nn

def count_trainable_params(model):
    """
    计算给定PyTorch模型的可训练参数数量。
    
    参数:
    model (nn.Module) - 要计算参数的PyTorch模型
    
    返回:
    int - 模型的可训练参数数量
    """
    trainable_params = 0
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params


model = AutoEncoder()
trainable_params = count_trainable_params(model)
print(f"Model has {trainable_params} trainable parameters.")

In [ ]:
trainable_params_AE = count_trainable_params(model)

In [ ]:
class AutoEncoder_1det(nn.Module):
    def __init__(self):
        super(AutoEncoder_1det, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(101, 20),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(20, 101),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
class WSClassifier_Onedetector(nn.Module):
    def __init__(self):
        super(WSClassifier_Onedetector, self).__init__()
        self.fc1 = nn.Linear(101, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
class WSClassifier(nn.Module):
    def __init__(self):
        super(WSClassifier, self).__init__()
        self.fc1 = nn.Linear(202, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
import torch.nn as nn

def count_trainable_params(model):
    """
    计算给定PyTorch模型的可训练参数数量。
    
    参数:
    model (nn.Module) - 要计算参数的PyTorch模型
    
    返回:
    int - 模型的可训练参数数量
    """
    trainable_params = 0
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params


model = WSClassifier()
trainable_params = count_trainable_params(model)
print(f"Model has {trainable_params} trainable parameters.")

In [ ]:
trainable_params_WSC = count_trainable_params(model)

In [ ]:
class WaveletCNNAE_xc(nn.Module):
    
    def __init__(
        self, 
        num_ifos: int,
        c_depth: int=8, 
        n_chann: int=64, 
        l1: int=1024
        # lx: int=200
    ):
        
        super(WaveletCNNAE_xc, self).__init__()
        
        self.c_depth = c_depth
        self.n_chann = n_chann
        
        self.cap_norm = nn.GroupNorm(num_ifos, num_ifos)
        
        self.Conv_In_encode = nn.Conv1d(
                in_channels=num_ifos, 
                out_channels=self.n_chann, 
                kernel_size=1
            )
        
        self.Conv_Out_encode = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=1, 
                kernel_size=1
            )
        
        self.Conv_In_decode = nn.Conv1d(
                in_channels=1, 
                out_channels=self.n_chann, 
                kernel_size=1
            )
        
        self.Conv_Out_decode = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=num_ifos, 
                kernel_size=1
            )
        
        self.body_norm_encode = nn.GroupNorm(4 ,n_chann)
        self.body_norm_decode = nn.GroupNorm(4 ,n_chann)
        self.end_norm_encode = nn.BatchNorm1d(1)
        self.end_norm_decode = nn.BatchNorm1d(1)
        
        self.WaveNet_layers_encode = nn.ModuleList()
        self.WaveNet_layers_decode = nn.ModuleList()
        self.WaveNet_layers_dp = nn.ModuleList()
        
        
        for i in range(self.c_depth):

            conv_layer = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=self.n_chann,
                kernel_size=2,
                dilation=2**i
            )
            
            self.WaveNet_layers_encode.append(conv_layer)
            
        for i in range(self.c_depth-1, -1, -1):

            conv_layer = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=self.n_chann,
                kernel_size=2,
                dilation=2**i
            )
            
            self.WaveNet_layers_decode.append(conv_layer)
            self.WaveNet_layers_dp.append(nn.ZeroPad1d(2**i))
        
        
#         self.Padding_layer = nn.ZeroPad1d(2**c_depth - 1)
                
        # self.L1 = nn.Linear(8192-2**c_depth, l1)
        
        # Consider replacing other batch normalizatoin layers with other nor method
        # Because batch norm are baised by the population of the CCSN rate in one batch 
        # This may produce overfitting model and will not be able to found at test phase
        # Question: Will we be able to figure out the side effect at infereceing phase?
                
#         self.conv_norm = nn.BatchNorm1d(200-2**c_depth + 1)
        self.L1 = nn.Linear(200-2**c_depth + 1, l1)
        self.L1_norm = nn.BatchNorm1d(l1)
        self.L2 = nn.Linear(l1, 200-2**c_depth + 1)
        self.L2_norm = nn.BatchNorm1d(200-2**c_depth + 1)

        nn.init.kaiming_normal_(self.Conv_In_encode.weight)
        nn.init.kaiming_normal_(self.Conv_Out_encode.weight)
        nn.init.constant_(self.Conv_In_encode.bias, 0.001)
        nn.init.constant_(self.Conv_Out_encode.bias, 0.001)
        
        nn.init.kaiming_normal_(self.Conv_In_decode.weight)
        nn.init.kaiming_normal_(self.Conv_Out_decode.weight)
        nn.init.constant_(self.Conv_In_decode.bias, 0.001)
        nn.init.constant_(self.Conv_Out_decode.bias, 0.001)

        # Initialize all the convolutional layer in between
        for conv_layer in self.WaveNet_layers_encode:
            nn.init.kaiming_normal_(conv_layer.weight)
            nn.init.constant_(conv_layer.bias, 0.001)
            
        for conv_layer in self.WaveNet_layers_decode:
            nn.init.kaiming_normal_(conv_layer.weight)
            nn.init.constant_(conv_layer.bias, 0.001)    

        nn.init.kaiming_uniform_(self.L1.weight)
        nn.init.kaiming_uniform_(self.L2.weight)
        nn.init.constant_(self.L1.bias, 0.001)
        nn.init.constant_(self.L2.bias, 0.001)
        
    def encode(self, x):
        
        x = self.cap_norm(x)
        x = self.Conv_In_encode(x)
        x = F.relu(x)
        
        # x = self.norm(x)
        
        for what_are_u_wavin_at in self.WaveNet_layers_encode:
            x = self.body_norm_encode(x)
            x = what_are_u_wavin_at(x)
            x = F.relu(x)
            
        x = self.Conv_Out_encode(x)
        x = F.relu(x)
        x = self.end_norm_encode(x)
        
        # print(x.shape)
        x = torch.flatten(x, 1)
        x = self.L1_norm(F.relu(self.L1(x)))
        
        # print('Encoder done')
        
        return x
    
    def decode(self, x):
        x = self.L2_norm(F.relu(self.L2(x)))
        
#         x = self.Padding_layer(x)
        
        x = torch.unsqueeze(x,1)

        # print(x.shape)
        
        # x = self.cap_norm(x)
        x = self.Conv_In_decode(x)
        x = F.relu(x)
        
        # x = self.norm(x)
        
        for (pad, dcd) in zip(self.WaveNet_layers_dp, self.WaveNet_layers_decode):
            # print(x.shape)
            x = self.body_norm_decode(x)
            x = pad(x)
            x = torch.flip(dcd(torch.flip(x, [-1])), [-1])
            x = F.relu(x)
        
        # print(x.shape)
        # print('CNN done')
        
        x = self.Conv_Out_decode(x)
        # print(x.shape)
        x = F.tanh(x)
        # print(x.shape)
        # x = self.end_norm_decode(x)
        
        # x = torch.flatten(x, 1)
        
        
        return x
    
    def forward(self,x):
        return self.decode(self.encode(x))

In [ ]:
def trainAE(dataset0_to_be_examined, cutID, version, datatype):
    
    if len(dataset0_to_be_examined) > 10 * trainable_params_AE:
        dataset = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_AE, replace = False)]
    else:
        dataset = dataset0_to_be_examined

    print('{} events passed to AE for training. '.format(len(dataset)))

    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainAE_Onedetector(dataset0_to_be_examined, cutID, version, datatype):
    
    if len(dataset0_to_be_examined) > 10 * trainable_params_AE:
        dataset = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_AE, replace = False)]
    else:
        dataset = dataset0_to_be_examined

    print('{} events passed to AE for training. '.format(len(dataset)))

    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder_1det().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainWSC(dataset0_to_be_examined, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    print('{} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1)))

    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

In [ ]:
def trainWSC_Onedetector(dataset0_to_be_examined, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    print('{} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1)))

    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier_Onedetector().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

#### Pass the first glith double WSL (pretrained)

#### Train the noise AE

#### Train the noise WSL

#### Train the BBH CNN

#### Train the BBH CNN WSL

#### Finally, let the test sample (WSL training sample) pass the procedure to get the label

#### Final label WSL

### Full pipeline

In [ ]:
version = 'v12'

#### First assume the CNN AE and the WSL is pretrained

In [ ]:
Ncut = 5;
cutList = {};

max_glitch_L = 0.0024;
min_glitch_L = 0.0008;
cutList["glitch_L"] = np.linspace(min_glitch_L, max_glitch_L, Ncut);

max_glitch_H = 0.0024;
min_glitch_H = 0.0004;
cutList["glitch_H"] = np.linspace(min_glitch_H, max_glitch_H, Ncut);

max_bkg = 0.0018;
min_bkg = 0.0008;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);


max_bbh_cnn = 0.0056
min_bbh_cnn = 0.0045
cutList['bbh_CNN'] = np.linspace(min_bbh_cnn, max_bbh_cnn, Ncut)

In [ ]:
# models = {}

In [ ]:
ind2datatype

In [ ]:
import itertools

In [ ]:
data_filtered

In [ ]:
del dataset_ae

In [ ]:
version

In [ ]:
cnt = 0;

ic = np.zeros(2, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if cnt < 23:
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    

        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_H']
    
    # --- Train the glitch AE for L

    models[previousStep+"_WSC_L"] = model_L
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
version = '15'

In [ ]:
cnt = 0;

ic = np.zeros(2, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if not np.all(ic == np.array([4,0])):
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    

        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_L']
    model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_H']
    
    # --- Train the glitch AE for L

    models[previousStep+"_WSC_L"] = model_L
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L >= 0, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L >= 0, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L >= 0, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
ic

In [ ]:
not np.all(ic == np.array([4,0,1,0]))

In [ ]:
cnt = 0;

ic = np.zeros(4, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    print(ic)
    
    if not np.all(ic == np.array([4,0,1,0])):
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    
    print('Starting training on ic = {}'.format(''.join(str(ic[j] for j in range(4)))))
    
        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    # dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']
    
    model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], cutID, version, 'glitch')
    models['glitch_L'] = model
    
    model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,101:], cutID, version, 'glitch')
    models['glitch_H'] = model
    
    dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()
    
    dataset0 = data_filtered['glitch'][:,:101]
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_L'][ic[2]]][:,:101]
    
    model_L = trainWSC_Onedetector(dataset0, dataset1, cutID, version, 'glitch')
    models[previousStep+"_WSC_L"] = model_L
    
    
    dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy()
    
    dataset0 = data_filtered['glitch'][:,101:]
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,101:]-dcd, axis=1) > cutList['glitch_H'][ic[3]]][:,101:]
    
    model_H = trainWSC_Onedetector(dataset0, dataset1, cutID, version, 'glitch')
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
dataset_wsl_filtered.shape

In [ ]:
(np.mean(np.var(dataset_wsl_collected[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]).shape

In [ ]:
signal_noise_like_args = np.argwhere(nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten() <= 0.5)

In [ ]:
signal_noise_like_args

In [ ]:
dataset_wsl_collected[signal_noise_like_args].shape

### Evaluating the pipeline data

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
sequence

In [ ]:
N_wsl.keys()

In [ ]:
correct_ans = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [2]*N_wsl['bbh'], [3]*N_wsl['sglf'], [4]*N_wsl['sghf']))
correct_ans_withoutsignal = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [-1]*(N_wsl['bbh']+N_wsl['sglf']+N_wsl['sghf'])))

In [ ]:
correct_ans_withoutsignal.shape

In [ ]:
list_datatype_withoutsignal = ['glitch','noise']

In [ ]:
testset = dataset_wsl_fft_collected

In [ ]:
version = 'v15'

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0
ic_withoutsignal = np.zeros(2, dtype = int)

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    dcd['noise_WSC'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    dcd['Final_WSC'] = nn.Sigmoid()(models["Final_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] <= 0.5, dcd['glitch_H'] <= 0.5))
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    # datatype = 'noise_WSC'
    datatype = 'Final_WSC'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
    listResult['ans'][cnt] = ans
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
version

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0
ic_withoutsignal = np.zeros(2, dtype = int)

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    if not np.all(ic_withoutsignal == np.array([4,0])):
        continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    dcd['noise_WSC'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    dcd['Final_WSC'] = nn.Sigmoid()(models["Final_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] < 0, dcd['glitch_H'] <= 0.5))
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    # datatype = 'noise_WSC'
    datatype = 'Final_WSC'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
    listResult['ans'][cnt] = ans
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
N_wsl['noise']

In [ ]:
listResult['ans']

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:] == 0)

In [ ]:
cnt = 20

np.sum(listResult['ans'][cnt][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:] == 1)

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][N_wsl['glitch']:N_wsl['glitch']+N_wsl['noise']] == 0)

In [ ]:
dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]].shape

In [ ]:
np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])

In [ ]:
version = 'v12'

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(['4','0']) + "_"+version+".json")

In [ ]:
dataset_wsl_fft.keys()

In [ ]:
model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['BBH_CNN_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v12

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v15

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
models

In [ ]:
# v14

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v14

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v14

model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
cutList['glitch_L']

In [ ]:
ic[2]

In [ ]:
# v14

model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'test')

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_L'][ic[2]], color="k", linestyle="--")
plt.legend()

In [ ]:
# v14

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_L'][ic[2]], color="k", linestyle="--")
plt.legend()

In [ ]:
# v14

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_H'][ic[3]], color="k", linestyle="--")
plt.legend()

### test training the glitch AE

In [ ]:
models = {}

In [ ]:
dataset_wsl_fft_filtered = dataset_wsl_fft_collected

In [ ]:
# model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'glitch')
# models['glitch_L'] = model

# model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,101:], 'test', 'test', 'glitch')
# models['glitch_H'] = model

dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_L'][1]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L


dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,101:]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,101:]-dcd, axis=1) > cutList['glitch_H'][0]][:,101:]

model_H = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_H"] = model_H

In [ ]:
dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft['noise'][np.var(dataset_wsl_fft['noise'][:,:101]-dcd, axis=1) > cutList['glitch_L'][3]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L

In [ ]:
# test

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,:101]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,:101] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Livinston noise")
    plt.axvline(cutList['glitch_L'][0], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_H'][0], color="k", linestyle="--")
plt.legend()

In [ ]:
# test

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,101:]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,101:] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Hanford noise")
    plt.axvline(cutList['glitch_H'][1], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test
model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()

## Remove the Livinston Glitch

### A quick check for roburtness

#### Data processing part

Setting hyperparameters

In [ ]:
N_wsl_total = 30000;
N_wsl = {}
N_wsl["noise"] = int(0.65*N_wsl_total)
N_wsl["bbh"] = int(0.1*N_wsl_total)
N_wsl["sglf"] = int(0.1*N_wsl_total)
N_wsl["sghf"] = int(0.1*N_wsl_total)
N_wsl["glitch"] = int(0.05*N_wsl_total)


snr_range = ['5-12','12-24','24-48','48-96']
ratio = [0.25, 0.25, 0.25, 0.25]
list_datatype = ["noise", "bbh", "sglf", "sghf", "glitch"]

In [ ]:
list_datatype_full = ['noise']

for snr in snr_range:
    list_datatype_full.append('bbh' + snr)
    list_datatype_full.append('sglf' + snr)
    list_datatype_full.append('sghf' + snr)
    
list_datatype_full.append('glitch')

In [ ]:
sequence = ['glitch','noise', 
            'bbh5-12', 'bbh12-24', 'bbh24-48', 'bbh48-96',
            'sglf5-12', 'sglf12-24', 'sglf24-48', 'sglf48-96',
            'sghf5-12', 'sghf12-24', 'sghf24-48', 'sghf48-96',]

In [ ]:
assert len(snr_range) == len(ratio)

for i in range(len(ratio)):

    N_wsl['bbh' + snr_range[i]] = int(ratio[i] * N_wsl['bbh'])
    
    N_wsl['sglf' + snr_range[i]] = int(ratio[i] * N_wsl['sglf'])
    
    N_wsl['sghf' + snr_range[i]] = int(ratio[i] * N_wsl['sghf'])

In [ ]:
### Introducing Chia-Jui's data

renorm_factor_0 = 20;
renorm_factor_1 = 20;

realbkg_L = np.load('../Data_cached/real_bkg_2202000_63917s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;
realbkg_H = np.load('../Data_cached/real_bkg_H_1466640_58803s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;

realbkg = np.concatenate((realbkg_L, realbkg_H), axis = 1).reshape(-1,200)

# realbbh = np.load('../Data_cached/injected_BBH_1823_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;

# realsg = np.load('../Data_cached/injected_lfsg_1835_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;
# realglitch = np.load("../data/real_glitches_9998_4000Hz_25ms.npz")["strain_time_data"]

realbbh_list = {}
realsglf_list = {}
realsghf_list = {}


for snr in snr_range:
    realbbh_list[snr] = np.load('../Data_cached/injected_BBH_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)
    realsglf_list[snr] = np.load('../Data_cached/injected_SGLF_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)
    realsghf_list[snr] = np.load('../Data_cached/injected_SGHF_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)

    

realglitch_L = np.load("../Data_cached/real_glitches_L_snrlt5_60132_4000Hz_50ms_centered.npz")["strain_time_data"][:50000].reshape(-1,1,200) / renorm_factor_1
realglitch_H = np.load('../Data_cached/real_glitches_H_snrlt5_59732_4000Hz_25ms.npz')["strain_time_data"][:50000].reshape(-1,1,200) / renorm_factor_1

Novel procedure. For training sample, the model is purely glitch trained. For testing and WSL sample, the model is one glitch + one noise

In [ ]:
num_glitch_for_AE = int(len(realglitch_L) * 0.8) - N_wsl['glitch'] // 2

In [ ]:
num_one_glitch_one_noise = int(len(realglitch_L) * 0.2) + N_wsl['glitch'] // 2

In [ ]:
np.random.shuffle(realglitch_L)
np.random.shuffle(realglitch_H)

In [ ]:
realglitch_L.shape

In [ ]:
realbkg.shape

In [ ]:
glitch_for_ae = np.concatenate((realglitch_L[:num_glitch_for_AE], realglitch_H[:num_glitch_for_AE]), axis=1).reshape(-1,200)
glitch_for_ae_fft = abs(np.fft.rfft(glitch_for_ae))
glitch_for_ae_fft = glitch_for_ae_fft/np.linalg.norm([glitch_for_ae_fft], axis=2).T

In [ ]:
glitch_for_ae = glitch_for_ae / np.linalg.norm([glitch_for_ae], axis = 2).T

glitch_for_ae = glitch_for_ae.reshape(-1,400)

In [ ]:
noise_for_glitch_building = realbkg[:2 * num_one_glitch_one_noise].reshape(-1,2,200)

# glitch_L_noise_H = np.concatenate((realglitch_L[-num_one_glitch_one_noise:], noise_for_glitch_building[:,[1],:]), axis = 1)
noise_L_glitch_H = np.concatenate((noise_for_glitch_building[:,[0],:], realglitch_H[-num_one_glitch_one_noise:]), axis = 1)
one_glitch_one_noise = noise_L_glitch_H
np.random.shuffle(one_glitch_one_noise)
one_glitch_one_noise = one_glitch_one_noise.reshape(-1,200)

realbkg = realbkg[2 * num_one_glitch_one_noise:]

In [ ]:
one_glitch_one_noise_fft = abs(np.fft.rfft(one_glitch_one_noise))
one_glitch_one_noise_fft = one_glitch_one_noise_fft / np.linalg.norm([one_glitch_one_noise_fft], axis=2).T

one_glitch_one_noise = one_glitch_one_noise / np.linalg.norm([one_glitch_one_noise], axis = 2).T

In [ ]:
one_glitch_one_noise_fft.shape

In [ ]:
realbkg_reserved = realbkg[-int(0.2 * len(realbkg)):]
# realbbh_reserved = realbbh[-int(0.2 * len(realbbh)):]
# realsg_reserved = realsg[-int(0.2 * len(realsg)):]
# realglitch_reserved = realglitch[-int(0.2 * len(realglitch)):]

realbkg = realbkg[:-int(0.2 * len(realbkg))]
# realbbh = realbbh[:-int(0.2 * len(realbbh))-int(0.2 * len(realbbh))%2]
# realsg = realsg[:-int(0.2 * len(realsg))-int(0.2 * len(realsg))%2]
# realglitch = realglitch[:-int(0.2 * len(realglitch))]

realbkg = realbkg / np.linalg.norm([realbkg], axis = 2).T

bkg_fft = abs(np.fft.rfft(realbkg))
bkg_fft = bkg_fft/np.linalg.norm([bkg_fft], axis=2).T

# bbh_fft = abs(np.fft.rfft(realbbh))
# bbh_fft = bbh_fft/np.linalg.norm([bbh_fft], axis=2).T

# sg_fft = abs(np.fft.rfft(realsg))
# sg_fft = sg_fft/np.linalg.norm([sg_fft], axis=2).T

# glitch_fft = abs(np.fft.rfft(realglitch))
# glitch_fft = glitch_fft/np.linalg.norm([glitch_fft], axis=2).T

In [ ]:
realbbh_reserved_list = {}
realsglf_reserved_list = {}
realsghf_reserved_list = {}

realbbh_fft_list = {}
realsglf_fft_list = {}
realsghf_fft_list = {}


for snr in snr_range:
    realbbh_reserved_list[snr] = realbbh_list[snr][-int(0.2 * len(realbbh_list[snr]))-int(0.2 * len(realbbh_list[snr]))%2:]
    realbbh_list[snr] = realbbh_list[snr][:-int(0.2 * len(realbbh_list[snr]))-int(0.2 * len(realbbh_list[snr]))%2]

    realbbh_fft_list[snr] = abs(np.fft.rfft(realbbh_list[snr]))
    realbbh_fft_list[snr] = realbbh_fft_list[snr]/np.linalg.norm([realbbh_fft_list[snr]], axis=2).T
    
    realbbh_list[snr] = realbbh_list[snr]/np.linalg.norm([realbbh_list[snr]], axis=2).T
    
    
    realsglf_reserved_list[snr] = realsglf_list[snr][-int(0.2 * len(realsglf_list[snr]))-int(0.2 * len(realsglf_list[snr]))%2:]
    realsglf_list[snr] = realsglf_list[snr][:-int(0.2 * len(realsglf_list[snr]))-int(0.2 * len(realsglf_list[snr]))%2]

    realsglf_fft_list[snr] = abs(np.fft.rfft(realsglf_list[snr]))
    realsglf_fft_list[snr] = realsglf_fft_list[snr]/np.linalg.norm([realsglf_fft_list[snr]], axis=2).T
    
    realsglf_list[snr] = realsglf_list[snr]/np.linalg.norm([realsglf_list[snr]], axis=2).T
    
    
    realsghf_reserved_list[snr] = realsghf_list[snr][-int(0.2 * len(realsghf_list[snr]))-int(0.2 * len(realsghf_list[snr]))%2:]
    realsghf_list[snr] = realsghf_list[snr][:-int(0.2 * len(realsghf_list[snr]))-int(0.2 * len(realsghf_list[snr]))%2]

    realsghf_fft_list[snr] = abs(np.fft.rfft(realsghf_list[snr]))
    realsghf_fft_list[snr] = realsghf_fft_list[snr]/np.linalg.norm([realsghf_fft_list[snr]], axis=2).T
    
    realsghf_list[snr] = realsghf_list[snr]/np.linalg.norm([realsghf_list[snr]], axis=2).T
    

In [ ]:
bkg_fft = bkg_fft.reshape(-1,202)

# bbh_fft = bbh_fft.reshape(-1,202)

# sg_fft = sg_fft.reshape(-1,202)

# glitch_fft = glitch_fft.reshape(-1,202)

glitch_for_ae_fft = glitch_for_ae_fft.reshape(-1,202)

one_glitch_one_noise_fft = one_glitch_one_noise_fft.reshape(-1,202)

In [ ]:
dataset_raw = {}
dataset_raw_fft = {}

dataset_raw["noise"] = realbkg.reshape(-1,400)
# dataset_raw_fft["bbh"] = bbh_fft
# dataset_raw_fft["sg"] = sg_fft
for snr in snr_range:
    dataset_raw['bbh' + snr] = realbbh_list[snr].reshape(-1,400)
    dataset_raw['sglf' + snr] = realsglf_list[snr].reshape(-1,400)
    dataset_raw['sghf' + snr] = realsghf_list[snr].reshape(-1,400)
# dataset_raw["glitch"] = glitch.reshape(-1,400)


dataset_raw_fft["noise"] = bkg_fft
# dataset_raw_fft["bbh"] = bbh_fft
# dataset_raw_fft["sg"] = sg_fft
for snr in snr_range:
    dataset_raw_fft['bbh' + snr] = realbbh_fft_list[snr].reshape(-1,202)
    dataset_raw_fft['sglf' + snr] = realsglf_fft_list[snr].reshape(-1,202)
    dataset_raw_fft['sghf' + snr] = realsghf_fft_list[snr].reshape(-1,202)
# dataset_raw_fft["glitch"] = glitch_fft

dataset_wsl = {};
dataset_ae = {};
dataset_wsl_fft = {};
dataset_ae_fft = {};

for dt in list_datatype_full[:-1]:
    perm = np.random.permutation(len(dataset_raw_fft[dt]))
    # perm = np.loadtxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_v7_GWAK.dat").astype(int)
    
    nwsl = N_wsl[dt]
    dataset_wsl[dt] = dataset_raw[dt][perm[:nwsl]]
    dataset_wsl_fft[dt] = dataset_raw_fft[dt][perm[:nwsl]]
    # dataset_wsl[dt] = dataset_wsl[dt] / np.linalg.norm([dataset_wsl[dt]], axis=2).T
    # dataset_wsl_fft[dt] = abs(np.fft.rfft(dataset_wsl[dt]))
    # dataset_wsl_fft[dt] = dataset_wsl_fft[dt]/np.linalg.norm([dataset_wsl_fft[dt]], axis=2).T
    
    dataset_ae[dt] = dataset_raw[dt][perm[nwsl:]]
    dataset_ae_fft[dt]  = dataset_raw_fft[dt][perm[nwsl:]]
    # dataset_ae[dt] = dataset_ae[dt] / np.linalg.norm([dataset_ae[dt]], axis=2).T
    # dataset_ae_fft[dt] = abs(np.fft.rfft(dataset_ae[dt]))
    # dataset_ae_fft[dt] = dataset_ae_fft[dt]/np.linalg.norm([dataset_ae_fft[dt]], axis=2).T
    
    # np.savetxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_"+version+"_2.dat", perm)
    
dataset_ae['bbh'] = np.zeros((0,400))
dataset_ae['sglf'] = np.zeros((0,400))
dataset_ae['sghf'] = np.zeros((0,400))

dataset_ae_fft['bbh'] = np.zeros((0,202))
dataset_ae_fft['sglf'] = np.zeros((0,202))
dataset_ae_fft['sghf'] = np.zeros((0,202))

for snr in snr_range:
    dataset_ae['bbh'] = np.append(dataset_ae['bbh'], dataset_ae['bbh'+snr])
    dataset_ae_fft['bbh'] = np.append(dataset_ae_fft['bbh'], dataset_ae_fft['bbh'+snr])
    
    dataset_ae['sglf'] = np.append(dataset_ae['sglf'], dataset_ae['sglf'+snr])
    dataset_ae_fft['sglf'] = np.append(dataset_ae_fft['sglf'], dataset_ae_fft['sglf'+snr])
    
    dataset_ae['sghf'] = np.append(dataset_ae['sghf'], dataset_ae['sghf'+snr])
    dataset_ae_fft['sghf'] = np.append(dataset_ae_fft['sghf'], dataset_ae_fft['sghf'+snr])

dataset_ae['bbh'] = dataset_ae['bbh'].reshape(-1,400)
dataset_ae['sglf'] = dataset_ae['sglf'].reshape(-1,400)
dataset_ae['sghf'] = dataset_ae['sghf'].reshape(-1,400)

dataset_ae_fft['bbh'] = dataset_ae_fft['bbh'].reshape(-1,202)
dataset_ae_fft['sglf'] = dataset_ae_fft['sglf'].reshape(-1,202)
dataset_ae_fft['sghf'] = dataset_ae_fft['sghf'].reshape(-1,202)

In [ ]:
dataset_raw[dt].shape

In [ ]:
dataset_wsl[dt]

In [ ]:
len(dataset_raw_fft[dt])

In [ ]:
dataset_wsl_fft['glitch'] = one_glitch_one_noise_fft[:N_wsl['glitch']]
dataset_wsl['glitch'] = one_glitch_one_noise[:2 * N_wsl['glitch']].reshape(-1,400)

dataset_ae_fft['glitch'] = glitch_for_ae_fft

# Missing dataset ae ['glitch'] here

In [ ]:
dataset_ae['glitch'] = glitch_for_ae

In [ ]:
dataset_wsl.keys()

In [ ]:
for key in dataset_wsl.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
for key in dataset_ae.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
for key in dataset_wsl_fft.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
dataset_wsl_fft.keys()

In [ ]:
dataset_ae_fft

In [ ]:
for key in dataset_ae.keys():
    print(key)
    print(dataset_ae[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_wsl.keys():
    print(key)
    print(dataset_wsl[key].shape)

In [ ]:
for key in dataset_wsl_fft.keys():
    print(key)
    print(dataset_wsl_fft[key].shape)

In [ ]:
dataset_wsl_fft_collected = np.empty((0, 202))
for dt in sequence:
    dataset_wsl_fft_collected = np.vstack((dataset_wsl_fft_collected, dataset_wsl_fft[dt]))

In [ ]:
dataset_wsl_collected = np.empty((0, 400))
for dt in sequence:
    dataset_wsl_collected = np.vstack((dataset_wsl_collected, dataset_wsl[dt]))

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
dataset_wsl_collected.shape

#### Model definition

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(202, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 202),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
import torch.nn as nn

def count_trainable_params(model):
    """
    计算给定PyTorch模型的可训练参数数量。
    
    参数:
    model (nn.Module) - 要计算参数的PyTorch模型
    
    返回:
    int - 模型的可训练参数数量
    """
    trainable_params = 0
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params


model = AutoEncoder()
trainable_params = count_trainable_params(model)
print(f"Model has {trainable_params} trainable parameters.")

In [ ]:
trainable_params_AE = count_trainable_params(model)

In [ ]:
class AutoEncoder_1det(nn.Module):
    def __init__(self):
        super(AutoEncoder_1det, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(101, 20),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(20, 101),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
class WSClassifier_Onedetector(nn.Module):
    def __init__(self):
        super(WSClassifier_Onedetector, self).__init__()
        self.fc1 = nn.Linear(101, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
class WSClassifier(nn.Module):
    def __init__(self):
        super(WSClassifier, self).__init__()
        self.fc1 = nn.Linear(202, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
import torch.nn as nn

def count_trainable_params(model):
    """
    计算给定PyTorch模型的可训练参数数量。
    
    参数:
    model (nn.Module) - 要计算参数的PyTorch模型
    
    返回:
    int - 模型的可训练参数数量
    """
    trainable_params = 0
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params


model = WSClassifier()
trainable_params = count_trainable_params(model)
print(f"Model has {trainable_params} trainable parameters.")

In [ ]:
trainable_params_WSC = count_trainable_params(model)

In [ ]:
class WaveletCNNAE_xc(nn.Module):
    
    def __init__(
        self, 
        num_ifos: int,
        c_depth: int=8, 
        n_chann: int=64, 
        l1: int=1024
        # lx: int=200
    ):
        
        super(WaveletCNNAE_xc, self).__init__()
        
        self.c_depth = c_depth
        self.n_chann = n_chann
        
        self.cap_norm = nn.GroupNorm(num_ifos, num_ifos)
        
        self.Conv_In_encode = nn.Conv1d(
                in_channels=num_ifos, 
                out_channels=self.n_chann, 
                kernel_size=1
            )
        
        self.Conv_Out_encode = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=1, 
                kernel_size=1
            )
        
        self.Conv_In_decode = nn.Conv1d(
                in_channels=1, 
                out_channels=self.n_chann, 
                kernel_size=1
            )
        
        self.Conv_Out_decode = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=num_ifos, 
                kernel_size=1
            )
        
        self.body_norm_encode = nn.GroupNorm(4 ,n_chann)
        self.body_norm_decode = nn.GroupNorm(4 ,n_chann)
        self.end_norm_encode = nn.BatchNorm1d(1)
        self.end_norm_decode = nn.BatchNorm1d(1)
        
        self.WaveNet_layers_encode = nn.ModuleList()
        self.WaveNet_layers_decode = nn.ModuleList()
        self.WaveNet_layers_dp = nn.ModuleList()
        
        
        for i in range(self.c_depth):

            conv_layer = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=self.n_chann,
                kernel_size=2,
                dilation=2**i
            )
            
            self.WaveNet_layers_encode.append(conv_layer)
            
        for i in range(self.c_depth-1, -1, -1):

            conv_layer = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=self.n_chann,
                kernel_size=2,
                dilation=2**i
            )
            
            self.WaveNet_layers_decode.append(conv_layer)
            self.WaveNet_layers_dp.append(nn.ZeroPad1d(2**i))
        
        
#         self.Padding_layer = nn.ZeroPad1d(2**c_depth - 1)
                
        # self.L1 = nn.Linear(8192-2**c_depth, l1)
        
        # Consider replacing other batch normalizatoin layers with other nor method
        # Because batch norm are baised by the population of the CCSN rate in one batch 
        # This may produce overfitting model and will not be able to found at test phase
        # Question: Will we be able to figure out the side effect at infereceing phase?
                
#         self.conv_norm = nn.BatchNorm1d(200-2**c_depth + 1)
        self.L1 = nn.Linear(200-2**c_depth + 1, l1)
        self.L1_norm = nn.BatchNorm1d(l1)
        self.L2 = nn.Linear(l1, 200-2**c_depth + 1)
        self.L2_norm = nn.BatchNorm1d(200-2**c_depth + 1)

        nn.init.kaiming_normal_(self.Conv_In_encode.weight)
        nn.init.kaiming_normal_(self.Conv_Out_encode.weight)
        nn.init.constant_(self.Conv_In_encode.bias, 0.001)
        nn.init.constant_(self.Conv_Out_encode.bias, 0.001)
        
        nn.init.kaiming_normal_(self.Conv_In_decode.weight)
        nn.init.kaiming_normal_(self.Conv_Out_decode.weight)
        nn.init.constant_(self.Conv_In_decode.bias, 0.001)
        nn.init.constant_(self.Conv_Out_decode.bias, 0.001)

        # Initialize all the convolutional layer in between
        for conv_layer in self.WaveNet_layers_encode:
            nn.init.kaiming_normal_(conv_layer.weight)
            nn.init.constant_(conv_layer.bias, 0.001)
            
        for conv_layer in self.WaveNet_layers_decode:
            nn.init.kaiming_normal_(conv_layer.weight)
            nn.init.constant_(conv_layer.bias, 0.001)    

        nn.init.kaiming_uniform_(self.L1.weight)
        nn.init.kaiming_uniform_(self.L2.weight)
        nn.init.constant_(self.L1.bias, 0.001)
        nn.init.constant_(self.L2.bias, 0.001)
        
    def encode(self, x):
        
        x = self.cap_norm(x)
        x = self.Conv_In_encode(x)
        x = F.relu(x)
        
        # x = self.norm(x)
        
        for what_are_u_wavin_at in self.WaveNet_layers_encode:
            x = self.body_norm_encode(x)
            x = what_are_u_wavin_at(x)
            x = F.relu(x)
            
        x = self.Conv_Out_encode(x)
        x = F.relu(x)
        x = self.end_norm_encode(x)
        
        # print(x.shape)
        x = torch.flatten(x, 1)
        x = self.L1_norm(F.relu(self.L1(x)))
        
        # print('Encoder done')
        
        return x
    
    def decode(self, x):
        x = self.L2_norm(F.relu(self.L2(x)))
        
#         x = self.Padding_layer(x)
        
        x = torch.unsqueeze(x,1)

        # print(x.shape)
        
        # x = self.cap_norm(x)
        x = self.Conv_In_decode(x)
        x = F.relu(x)
        
        # x = self.norm(x)
        
        for (pad, dcd) in zip(self.WaveNet_layers_dp, self.WaveNet_layers_decode):
            # print(x.shape)
            x = self.body_norm_decode(x)
            x = pad(x)
            x = torch.flip(dcd(torch.flip(x, [-1])), [-1])
            x = F.relu(x)
        
        # print(x.shape)
        # print('CNN done')
        
        x = self.Conv_Out_decode(x)
        # print(x.shape)
        x = F.tanh(x)
        # print(x.shape)
        # x = self.end_norm_decode(x)
        
        # x = torch.flatten(x, 1)
        
        
        return x
    
    def forward(self,x):
        return self.decode(self.encode(x))

In [ ]:
def trainAE(dataset0_to_be_examined, cutID, version, datatype):
    
    if len(dataset0_to_be_examined) > 10 * trainable_params_AE:
        dataset = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_AE, replace = False)]
    else:
        dataset = dataset0_to_be_examined

    print('{} events passed to AE for training. '.format(len(dataset)))

    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainAE_Onedetector(dataset0_to_be_examined, cutID, version, datatype):
    
    if len(dataset0_to_be_examined) > 10 * trainable_params_AE:
        dataset = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_AE, replace = False)]
    else:
        dataset = dataset0_to_be_examined

    print('{} events passed to AE for training. '.format(len(dataset)))

    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder_1det().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainWSC(dataset0_to_be_examined, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    print('{} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1)))

    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

In [ ]:
def trainWSC_Onedetector(dataset0_to_be_examined, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    print('{} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1)))

    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier_Onedetector().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

#### Pass the first glith double WSL (pretrained)

#### Train the noise AE

#### Train the noise WSL

#### Train the BBH CNN

#### Train the BBH CNN WSL

#### Finally, let the test sample (WSL training sample) pass the procedure to get the label

#### Final label WSL

### Full pipeline

In [ ]:
version = 'v16'

#### First assume the CNN AE and the WSL is pretrained

In [ ]:
Ncut = 5;
cutList = {};

max_glitch_L = 0.0024;
min_glitch_L = 0.0008;
cutList["glitch_L"] = np.linspace(min_glitch_L, max_glitch_L, Ncut);

max_glitch_H = 0.0024;
min_glitch_H = 0.0004;
cutList["glitch_H"] = np.linspace(min_glitch_H, max_glitch_H, Ncut);

max_bkg = 0.0018;
min_bkg = 0.0008;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);


max_bbh_cnn = 0.0056
min_bbh_cnn = 0.0045
cutList['bbh_CNN'] = np.linspace(min_bbh_cnn, max_bbh_cnn, Ncut)

In [ ]:
# models = {}

In [ ]:
ind2datatype

In [ ]:
import itertools

In [ ]:
data_filtered

In [ ]:
del dataset_ae

In [ ]:
version

In [ ]:
epochs = 50
epochs_wsl = 50

In [ ]:
cnt = 0;

ic = np.zeros(2, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if cnt < 23:
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    

        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_H']
    
    # --- Train the glitch AE for L

    models[previousStep+"_WSC_L"] = model_L
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
version = '15'

In [ ]:
models = {}

In [ ]:
cnt = 0;

ic = np.zeros(2, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if not np.all(ic == np.array([4,0])):
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    

        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_L']
    model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_H']
    
    # --- Train the glitch AE for L

    models[previousStep+"_WSC_L"] = model_L
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L >= 0, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L >= 0, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L >= 0, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
ic

In [ ]:
not np.all(ic == np.array([4,0,1,0]))

In [ ]:
cnt = 0;

ic = np.zeros(4, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    print(ic)
    
    if not np.all(ic == np.array([4,0,1,0])):
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    
    print('Starting training on ic = {}'.format(''.join(str(ic[j] for j in range(4)))))
    
        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    # dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']
    
    model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], cutID, version, 'glitch')
    models['glitch_L'] = model
    
    model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,101:], cutID, version, 'glitch')
    models['glitch_H'] = model
    
    dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()
    
    dataset0 = data_filtered['glitch'][:,:101]
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_L'][ic[2]]][:,:101]
    
    model_L = trainWSC_Onedetector(dataset0, dataset1, cutID, version, 'glitch')
    models[previousStep+"_WSC_L"] = model_L
    
    
    dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy()
    
    dataset0 = data_filtered['glitch'][:,101:]
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,101:]-dcd, axis=1) > cutList['glitch_H'][ic[3]]][:,101:]
    
    model_H = trainWSC_Onedetector(dataset0, dataset1, cutID, version, 'glitch')
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
dataset_wsl_filtered.shape

In [ ]:
(np.mean(np.var(dataset_wsl_collected[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]).shape

In [ ]:
signal_noise_like_args = np.argwhere(nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten() <= 0.5)

In [ ]:
signal_noise_like_args

In [ ]:
dataset_wsl_collected[signal_noise_like_args].shape

### Evaluating the pipeline data

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
sequence

In [ ]:
N_wsl.keys()

In [ ]:
correct_ans = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [2]*N_wsl['bbh'], [3]*N_wsl['sglf'], [4]*N_wsl['sghf']))
correct_ans_withoutsignal = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [-1]*(N_wsl['bbh']+N_wsl['sglf']+N_wsl['sghf'])))

In [ ]:
correct_ans_withoutsignal.shape

In [ ]:
list_datatype_withoutsignal = ['glitch','noise']

In [ ]:
testset = dataset_wsl_fft_collected

In [ ]:
version = 'v15'

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0
ic_withoutsignal = np.zeros(2, dtype = int)

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    dcd['noise_WSC'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    dcd['Final_WSC'] = nn.Sigmoid()(models["Final_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] <= 0.5, dcd['glitch_H'] <= 0.5))
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    # datatype = 'noise_WSC'
    datatype = 'Final_WSC'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
    listResult['ans'][cnt] = ans
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
version

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0
ic_withoutsignal = np.zeros(2, dtype = int)

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    if not np.all(ic_withoutsignal == np.array([4,0])):
        continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.ones(len(testset), dtype="int") * 2
    
    dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    dcd['noise_WSC'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    dcd['Final_WSC'] = nn.Sigmoid()(models["Final_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] < 0, dcd['glitch_H'] <= 0.5))
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    # datatype = 'noise_WSC'
    datatype = 'Final_WSC'
    # ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    # print(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]].shape)
    # print(int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]))))
    # print(np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    
    # ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5)
    ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
    listResult['ans'][cnt] = ans
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
N_wsl['noise']

In [ ]:
listResult['ans']

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:] == 0)

In [ ]:
cnt = 20

np.sum(listResult['ans'][cnt][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:] == 1)

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][N_wsl['glitch']:N_wsl['glitch']+N_wsl['noise']] == 1)

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][:N_wsl['glitch']] == 0)

In [ ]:
dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]].shape

In [ ]:
np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])

In [ ]:
version = 'v12'

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(['4','0']) + "_"+version+".json")

In [ ]:
dataset_wsl_fft.keys()

In [ ]:
model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of noise WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['BBH_CNN_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of BBH_CNN WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v12

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v15

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','4']) + '_v2.json')['glitch_L']
model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','4']) + '_v2.json')['glitch_H']

In [ ]:
# Casual Plot

model = model_L

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L, dataset L")
        
        
    # for snr in snr_range:

    #     key = signal_type+snr
    #     dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
    #     foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# Casual Plot

model = model_L

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L, dataset H")
        
        
    # for snr in snr_range:

    #     key = signal_type+snr
    #     dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
    #     foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# Casual Plot

model = model_H

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H, dataset H")
        
        
    # for snr in snr_range:

    #     key = signal_type+snr
    #     dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
    #     foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# Casual Plot

model = model_H

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H, dataset L")
        
        
    # for snr in snr_range:

    #     key = signal_type+snr
    #     dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
    #     foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

count = 0


for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)
        
        count += np.sum(dcd_signal < 0.5)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
count

In [ ]:
models

In [ ]:
# v14

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v14

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v14

model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
cutList['glitch_L']

In [ ]:
ic[2]

In [ ]:
# v14

model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'test')

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_L'][ic[2]], color="k", linestyle="--")
plt.legend()

In [ ]:
# v14

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_L'][ic[2]], color="k", linestyle="--")
plt.legend()

In [ ]:
# v14

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_H'][ic[3]], color="k", linestyle="--")
plt.legend()

### test training the glitch AE

In [ ]:
models = {}

In [ ]:
dataset_wsl_fft_filtered = dataset_wsl_fft_collected

In [ ]:
# model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'glitch')
# models['glitch_L'] = model

# model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,101:], 'test', 'test', 'glitch')
# models['glitch_H'] = model

dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_L'][1]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L


dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,101:]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,101:]-dcd, axis=1) > cutList['glitch_H'][0]][:,101:]

model_H = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_H"] = model_H

In [ ]:
dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft['noise'][np.var(dataset_wsl_fft['noise'][:,:101]-dcd, axis=1) > cutList['glitch_L'][0]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L

In [ ]:
# Don't use this. This is not strict as we are using Hanford glitch model to fit for the Livinston

dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_H'][0]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L

In [ ]:
model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'glitch')
models['glitch_L'] = model

In [ ]:
# test

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,:101]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,:101] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Livinston noise")
    # plt.axvline(cutList['glitch_L'][0], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,:101]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,:101] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Livinston noise")
    plt.axvline(cutList['glitch_L'][0], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test
# Be careful here. The AE is using the Hanford AE

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test
# Be careful here. The AE is using the Hanford AE

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test
# Be careful here. The AE is using the Hanford AE

model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_H'][0], color="k", linestyle="--")
plt.legend()

In [ ]:
models

In [ ]:
# test
# How about Livinston data in Hanford model

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,:101]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,:101] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Hanford glitch")
    plt.axvline(cutList['glitch_H'][1], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,101:]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,101:] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Hanford noise")
    plt.axvline(cutList['glitch_H'][1], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test
model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()

### How about the performance for glitch trained CNN AE?

In [ ]:
model_CNN = torch.load('../Model_cached/CNN_GLITCH/model_dep_6_chnl_12_btn_20.pt')

In [ ]:
dcd_glitch = model(torch.FloatTensor(dataset_ae['glitch'][:10000][:,:200].reshape(-1,1,200)))

In [ ]:
np.var(dcd_glitch.detach().numpy() - dataset_ae['glitch'][:10000][:,:200].reshape(-1,1,200), axis = -1)

In [ ]:
dcd_bkg = model(torch.FloatTensor(dataset_wsl['noise'][:,:200].reshape(-1,1,200))).detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae['glitch'][:10000][:,:200].reshape(-1,1,200))).detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl['noise'][:,:200].reshape(-1,1,200)-dcd_bkg, axis=-1).flatten(), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae['glitch'][:10000][:,:200].reshape(-1,1,200)-dcd_glitch, axis=-1).flatten(), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl[key_to_plot][:,:200].reshape(-1,1,200))).detach().numpy()
        plt.hist(np.var(dataset_wsl[key_to_plot][:,:200].reshape(-1,1,200) - dcd_signal, axis = -1).flatten(), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Livinston glitch")
    # plt.axvline(cutList['glitch_H'][1], color="k", linestyle="--")
    # plt.axvline(, color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
dataset_wsl_collected[:,:200].shape

In [ ]:
dcd = model_CNN(torch.FloatTensor(dataset_wsl_collected[:,:200].reshape(-1,1,200))).detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft_collected[np.var(dataset_wsl_collected[:,:200].reshape(-1,1,200)-dcd, axis=-1).flatten()>0.004][:,:101]

In [ ]:
model = trainWSC_Onedetector(dataset1, dataset0, 'test', 'test', 'test')

In [ ]:
bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_ae_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = nn.Sigmoid()(model(torch.FloatTensor(glitch_filtered))).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

### Make a 3-class learning

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(['4','0']) + "_"+'v12'+".json")

In [ ]:
models

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
dcd = {}

dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()

for type in ['noise_WSC', 'BBH_CNN_WSC']:
    dcd[type] = nn.Sigmoid()(models[type](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

In [ ]:
dcd['glitch_WSC_L'].min()

In [ ]:
glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)
noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

pass_idx = np.logical_and(glitch_pass, noise_pass)

signal_events = dataset_wsl_fft_collected[pass_idx]

In [ ]:
signal_events.shape

In [ ]:
dataset_ae_fft.keys()

In [ ]:
model_3class = trainWSC_3class(dataset_ae_fft['glitch_oneglitch_onenoise'], dataset_ae_fft['noise'], signal_events, 'test', 'test', 'test')

## Combined WSL

### A quick check for roburtness

#### Data processing part

Setting hyperparameters

In [ ]:
epochs = 50

In [ ]:
N_wsl_total = 30000;
N_wsl = {}
N_wsl["noise"] = int(0.65*N_wsl_total)
N_wsl["bbh"] = int(0.1*N_wsl_total)
N_wsl["sglf"] = int(0.1*N_wsl_total)
N_wsl["sghf"] = int(0.1*N_wsl_total)
N_wsl["glitch"] = int(0.05*N_wsl_total)


snr_range = ['5-12','12-24','24-48','48-96']
ratio = [0.25, 0.25, 0.25, 0.25]
list_datatype = ["noise", "bbh", "sglf", "sghf", "glitch"]

In [ ]:
list_datatype_full = ['noise']

for snr in snr_range:
    list_datatype_full.append('bbh' + snr)
    list_datatype_full.append('sglf' + snr)
    list_datatype_full.append('sghf' + snr)
    
list_datatype_full.append('glitch')

In [ ]:
sequence = ['glitch','noise', 
            'bbh5-12', 'bbh12-24', 'bbh24-48', 'bbh48-96',
            'sglf5-12', 'sglf12-24', 'sglf24-48', 'sglf48-96',
            'sghf5-12', 'sghf12-24', 'sghf24-48', 'sghf48-96',]

In [ ]:
assert len(snr_range) == len(ratio)

for i in range(len(ratio)):

    N_wsl['bbh' + snr_range[i]] = int(ratio[i] * N_wsl['bbh'])
    
    N_wsl['sglf' + snr_range[i]] = int(ratio[i] * N_wsl['sglf'])
    
    N_wsl['sghf' + snr_range[i]] = int(ratio[i] * N_wsl['sghf'])

In [ ]:
### Introducing Chia-Jui's data

renorm_factor_0 = 20;
renorm_factor_1 = 20;

realbkg_L = np.load('../Data_cached/real_bkg_2202000_63917s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;
realbkg_H = np.load('../Data_cached/real_bkg_H_1466640_58803s_4000Hz_50ms.npy')[:1000000].reshape(-1,1,200) / renorm_factor_0;

realbkg = np.concatenate((realbkg_L, realbkg_H), axis = 1).reshape(-1,200)

# realbbh = np.load('../Data_cached/injected_BBH_1823_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;

# realsg = np.load('../Data_cached/injected_lfsg_1835_around_merger_time_63917_58803.npz')['strain'].reshape(-1,200) / renorm_factor_0;
# realglitch = np.load("../data/real_glitches_9998_4000Hz_25ms.npz")["strain_time_data"]

realbbh_list = {}
realsglf_list = {}
realsghf_list = {}


for snr in snr_range:
    realbbh_list[snr] = np.load('../Data_cached/injected_BBH_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)
    realsglf_list[snr] = np.load('../Data_cached/injected_SGLF_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)
    realsghf_list[snr] = np.load('../Data_cached/injected_SGHF_55k_snr{}_0th_events_before_merger_time_windowlength_200.npz'.format(snr))['strain'][:25000].reshape(-1,200)

    

realglitch_L = np.load("../Data_cached/real_glitches_snrlt5_60132_4000Hz_25ms.npz")["strain_time_data"][:50000].reshape(-1,1,200) / renorm_factor_1
realglitch_H = np.load('../Data_cached/real_glitches_H_snrlt5_59732_4000Hz_25ms.npz')["strain_time_data"][:50000].reshape(-1,1,200) / renorm_factor_1

Novel procedure. For training sample, the model is purely glitch trained. For testing and WSL sample, the model is one glitch + one noise

In [ ]:
num_glitch_for_AE = int(len(realglitch_L) * 0.8) - N_wsl['glitch'] // 2

In [ ]:
num_one_glitch_one_noise = int(len(realglitch_L) * 0.2) + N_wsl['glitch'] // 2

In [ ]:
np.random.shuffle(realglitch_L)
np.random.shuffle(realglitch_H)

In [ ]:
realglitch_L.shape

In [ ]:
realbkg.shape

In [ ]:
glitch_for_ae = np.concatenate((realglitch_L[:num_glitch_for_AE], realglitch_H[:num_glitch_for_AE]), axis=1).reshape(-1,200)
glitch_for_ae_fft = abs(np.fft.rfft(glitch_for_ae))
glitch_for_ae_fft = glitch_for_ae_fft/np.linalg.norm([glitch_for_ae_fft], axis=2).T

In [ ]:
noise_for_glitch_building = realbkg[:2 * num_one_glitch_one_noise].reshape(-1,2,200)

glitch_L_noise_H = np.concatenate((realglitch_L[-num_one_glitch_one_noise:], noise_for_glitch_building[:,[1],:]), axis = 1)
noise_L_glitch_H = np.concatenate((noise_for_glitch_building[:,[0],:], realglitch_H[-num_one_glitch_one_noise:]), axis = 1)
one_glitch_one_noise = np.vstack((glitch_L_noise_H, noise_L_glitch_H))
np.random.shuffle(one_glitch_one_noise)
one_glitch_one_noise = one_glitch_one_noise.reshape(-1,200)

realbkg = realbkg[2 * num_one_glitch_one_noise:]

In [ ]:
one_glitch_one_noise_fft = abs(np.fft.rfft(one_glitch_one_noise))
one_glitch_one_noise_fft = one_glitch_one_noise_fft / np.linalg.norm([one_glitch_one_noise_fft], axis=2).T

one_glitch_one_noise = one_glitch_one_noise / np.linalg.norm([one_glitch_one_noise], axis = 2).T

In [ ]:
realbkg_reserved = realbkg[-int(0.2 * len(realbkg)):]
# realbbh_reserved = realbbh[-int(0.2 * len(realbbh)):]
# realsg_reserved = realsg[-int(0.2 * len(realsg)):]
# realglitch_reserved = realglitch[-int(0.2 * len(realglitch)):]

realbkg = realbkg[:-int(0.2 * len(realbkg))]
# realbbh = realbbh[:-int(0.2 * len(realbbh))-int(0.2 * len(realbbh))%2]
# realsg = realsg[:-int(0.2 * len(realsg))-int(0.2 * len(realsg))%2]
# realglitch = realglitch[:-int(0.2 * len(realglitch))]

realbkg = realbkg / np.linalg.norm([realbkg], axis = 2).T

bkg_fft = abs(np.fft.rfft(realbkg))
bkg_fft = bkg_fft/np.linalg.norm([bkg_fft], axis=2).T

# bbh_fft = abs(np.fft.rfft(realbbh))
# bbh_fft = bbh_fft/np.linalg.norm([bbh_fft], axis=2).T

# sg_fft = abs(np.fft.rfft(realsg))
# sg_fft = sg_fft/np.linalg.norm([sg_fft], axis=2).T

# glitch_fft = abs(np.fft.rfft(realglitch))
# glitch_fft = glitch_fft/np.linalg.norm([glitch_fft], axis=2).T

In [ ]:
realbbh_reserved_list = {}
realsglf_reserved_list = {}
realsghf_reserved_list = {}

realbbh_fft_list = {}
realsglf_fft_list = {}
realsghf_fft_list = {}


for snr in snr_range:
    realbbh_reserved_list[snr] = realbbh_list[snr][-int(0.2 * len(realbbh_list[snr]))-int(0.2 * len(realbbh_list[snr]))%2:]
    realbbh_list[snr] = realbbh_list[snr][:-int(0.2 * len(realbbh_list[snr]))-int(0.2 * len(realbbh_list[snr]))%2]

    realbbh_fft_list[snr] = abs(np.fft.rfft(realbbh_list[snr]))
    realbbh_fft_list[snr] = realbbh_fft_list[snr]/np.linalg.norm([realbbh_fft_list[snr]], axis=2).T
    
    realbbh_list[snr] = realbbh_list[snr]/np.linalg.norm([realbbh_list[snr]], axis=2).T
    
    
    realsglf_reserved_list[snr] = realsglf_list[snr][-int(0.2 * len(realsglf_list[snr]))-int(0.2 * len(realsglf_list[snr]))%2:]
    realsglf_list[snr] = realsglf_list[snr][:-int(0.2 * len(realsglf_list[snr]))-int(0.2 * len(realsglf_list[snr]))%2]

    realsglf_fft_list[snr] = abs(np.fft.rfft(realsglf_list[snr]))
    realsglf_fft_list[snr] = realsglf_fft_list[snr]/np.linalg.norm([realsglf_fft_list[snr]], axis=2).T
    
    realsglf_list[snr] = realsglf_list[snr]/np.linalg.norm([realsglf_list[snr]], axis=2).T
    
    
    realsghf_reserved_list[snr] = realsghf_list[snr][-int(0.2 * len(realsghf_list[snr]))-int(0.2 * len(realsghf_list[snr]))%2:]
    realsghf_list[snr] = realsghf_list[snr][:-int(0.2 * len(realsghf_list[snr]))-int(0.2 * len(realsghf_list[snr]))%2]

    realsghf_fft_list[snr] = abs(np.fft.rfft(realsghf_list[snr]))
    realsghf_fft_list[snr] = realsghf_fft_list[snr]/np.linalg.norm([realsghf_fft_list[snr]], axis=2).T
    
    realsghf_list[snr] = realsghf_list[snr]/np.linalg.norm([realsghf_list[snr]], axis=2).T
    

In [ ]:
bkg_fft = bkg_fft.reshape(-1,202)

# bbh_fft = bbh_fft.reshape(-1,202)

# sg_fft = sg_fft.reshape(-1,202)

# glitch_fft = glitch_fft.reshape(-1,202)

glitch_for_ae_fft = glitch_for_ae_fft.reshape(-1,202)

one_glitch_one_noise_fft = one_glitch_one_noise_fft.reshape(-1,202)

In [ ]:
dataset_raw = {}
dataset_raw_fft = {}

dataset_raw["noise"] = realbkg.reshape(-1,400)
# dataset_raw_fft["bbh"] = bbh_fft
# dataset_raw_fft["sg"] = sg_fft
for snr in snr_range:
    dataset_raw['bbh' + snr] = realbbh_list[snr].reshape(-1,400)
    dataset_raw['sglf' + snr] = realsglf_list[snr].reshape(-1,400)
    dataset_raw['sghf' + snr] = realsghf_list[snr].reshape(-1,400)
# dataset_raw["glitch"] = glitch.reshape(-1,400)


dataset_raw_fft["noise"] = bkg_fft
# dataset_raw_fft["bbh"] = bbh_fft
# dataset_raw_fft["sg"] = sg_fft
for snr in snr_range:
    dataset_raw_fft['bbh' + snr] = realbbh_fft_list[snr].reshape(-1,202)
    dataset_raw_fft['sglf' + snr] = realsglf_fft_list[snr].reshape(-1,202)
    dataset_raw_fft['sghf' + snr] = realsghf_fft_list[snr].reshape(-1,202)
# dataset_raw_fft["glitch"] = glitch_fft

dataset_wsl = {};
dataset_ae = {};
dataset_wsl_fft = {};
dataset_ae_fft = {};

for dt in list_datatype_full[:-1]:
    perm = np.random.permutation(len(dataset_raw_fft[dt]))
    # perm = np.loadtxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_v7_GWAK.dat").astype(int)
    
    nwsl = N_wsl[dt]
    dataset_wsl[dt] = dataset_raw[dt][perm[:nwsl]]
    dataset_wsl_fft[dt] = dataset_raw_fft[dt][perm[:nwsl]]
    # dataset_wsl[dt] = dataset_wsl[dt] / np.linalg.norm([dataset_wsl[dt]], axis=2).T
    # dataset_wsl_fft[dt] = abs(np.fft.rfft(dataset_wsl[dt]))
    # dataset_wsl_fft[dt] = dataset_wsl_fft[dt]/np.linalg.norm([dataset_wsl_fft[dt]], axis=2).T
    
    dataset_ae[dt] = dataset_raw[dt][perm[nwsl:]]
    dataset_ae_fft[dt]  = dataset_raw_fft[dt][perm[nwsl:]]
    # dataset_ae[dt] = dataset_ae[dt] / np.linalg.norm([dataset_ae[dt]], axis=2).T
    # dataset_ae_fft[dt] = abs(np.fft.rfft(dataset_ae[dt]))
    # dataset_ae_fft[dt] = dataset_ae_fft[dt]/np.linalg.norm([dataset_ae_fft[dt]], axis=2).T
    
    # np.savetxt("../Data_Cached/SequentialTraining/WSL/perm_"+dt+"_2det_Chia-Jui_"+version+"_2.dat", perm)
    
dataset_ae['bbh'] = np.zeros((0,400))
dataset_ae['sglf'] = np.zeros((0,400))
dataset_ae['sghf'] = np.zeros((0,400))

dataset_ae_fft['bbh'] = np.zeros((0,202))
dataset_ae_fft['sglf'] = np.zeros((0,202))
dataset_ae_fft['sghf'] = np.zeros((0,202))

for snr in snr_range:
    dataset_ae['bbh'] = np.append(dataset_ae['bbh'], dataset_ae['bbh'+snr])
    dataset_ae_fft['bbh'] = np.append(dataset_ae_fft['bbh'], dataset_ae_fft['bbh'+snr])
    
    dataset_ae['sglf'] = np.append(dataset_ae['sglf'], dataset_ae['sglf'+snr])
    dataset_ae_fft['sglf'] = np.append(dataset_ae_fft['sglf'], dataset_ae_fft['sglf'+snr])
    
    dataset_ae['sghf'] = np.append(dataset_ae['sghf'], dataset_ae['sghf'+snr])
    dataset_ae_fft['sghf'] = np.append(dataset_ae_fft['sghf'], dataset_ae_fft['sghf'+snr])

dataset_ae['bbh'] = dataset_ae['bbh'].reshape(-1,400)
dataset_ae['sglf'] = dataset_ae['sglf'].reshape(-1,400)
dataset_ae['sghf'] = dataset_ae['sghf'].reshape(-1,400)

dataset_ae_fft['bbh'] = dataset_ae_fft['bbh'].reshape(-1,202)
dataset_ae_fft['sglf'] = dataset_ae_fft['sglf'].reshape(-1,202)
dataset_ae_fft['sghf'] = dataset_ae_fft['sghf'].reshape(-1,202)

In [ ]:
dataset_raw[dt].shape

In [ ]:
dataset_wsl[dt]

In [ ]:
len(dataset_raw_fft[dt])

In [ ]:
dataset_wsl_fft['glitch'] = one_glitch_one_noise_fft[:N_wsl['glitch']]
dataset_wsl['glitch'] = one_glitch_one_noise[:2 * N_wsl['glitch']].reshape(-1,400)

dataset_ae_fft['glitch'] = glitch_for_ae_fft

dataset_ae_fft['glitch_oneglitch_onenoise'] = one_glitch_one_noise_fft[N_wsl['glitch']:]

# Missing dataset ae ['glitch'] here

In [ ]:
dataset_wsl.keys()

In [ ]:
for key in dataset_wsl.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
for key in dataset_ae.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
for key in dataset_wsl_fft.keys():

    print(np.linalg.norm(dataset_wsl[key][0]))

In [ ]:
dataset_wsl_fft.keys()

In [ ]:
dataset_ae_fft

In [ ]:
for key in dataset_ae.keys():
    print(key)
    print(dataset_ae[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_ae_fft.keys():
    print(key)
    print(dataset_ae_fft[key].shape)

In [ ]:
for key in dataset_wsl.keys():
    print(key)
    print(dataset_wsl[key].shape)

In [ ]:
for key in dataset_wsl_fft.keys():
    print(key)
    print(dataset_wsl_fft[key].shape)

In [ ]:
dataset_wsl_fft_collected = np.empty((0, 202))
for dt in sequence:
    dataset_wsl_fft_collected = np.vstack((dataset_wsl_fft_collected, dataset_wsl_fft[dt]))

In [ ]:
dataset_wsl_collected = np.empty((0, 400))
for dt in sequence:
    dataset_wsl_collected = np.vstack((dataset_wsl_collected, dataset_wsl[dt]))

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
dataset_wsl_collected.shape

#### Model definition

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(202, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 202),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
import torch.nn as nn

def count_trainable_params(model):
    """
    计算给定PyTorch模型的可训练参数数量。
    
    参数:
    model (nn.Module) - 要计算参数的PyTorch模型
    
    返回:
    int - 模型的可训练参数数量
    """
    trainable_params = 0
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params


model = AutoEncoder()
trainable_params = count_trainable_params(model)
print(f"Model has {trainable_params} trainable parameters.")

In [ ]:
trainable_params_AE = count_trainable_params(model)

In [ ]:
class AutoEncoder_1det(nn.Module):
    def __init__(self):
        super(AutoEncoder_1det, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(101, 20),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(20, 101),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
class WSClassifier_Onedetector(nn.Module):
    def __init__(self):
        super(WSClassifier_Onedetector, self).__init__()
        self.fc1 = nn.Linear(101, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
class WSClassifier(nn.Module):
    def __init__(self):
        super(WSClassifier, self).__init__()
        self.fc1 = nn.Linear(202, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 1)
        # self.norm2 = nn.BatchNorm1d(8)
        # self.fc4 = nn.Linear(8, 1)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm1(self.relu(self.fc1(x)))
        return self.fc2(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
class WSClassifier_3class(nn.Module):
    def __init__(self):
        super(WSClassifier_3class, self).__init__()
        self.fc1 = nn.Linear(202, 32)  # 第一层全连接层，输入维度为4，输出维度为64
        self.norm1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()  # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(32, 8)
        self.norm2 = nn.BatchNorm1d(8)
        self.fc4 = nn.Linear(8, 3)  # 第三层全连接层，输入维度为32，输出维度为类别数目
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = self.norm2(self.relu(self.fc2(self.norm1(self.relu(self.fc1(x))))))
        return self.fc4(x)
        # x = self.relu(x)
#         x = self.sigmoid(x)

In [ ]:
import torch.nn as nn

def count_trainable_params(model):
    """
    计算给定PyTorch模型的可训练参数数量。
    
    参数:
    model (nn.Module) - 要计算参数的PyTorch模型
    
    返回:
    int - 模型的可训练参数数量
    """
    trainable_params = 0
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params


model = WSClassifier_3class()
trainable_params = count_trainable_params(model)
print(f"Model has {trainable_params} trainable parameters.")

In [ ]:
trainable_params_WSC = count_trainable_params(model)

In [ ]:
class WaveletCNNAE_xc(nn.Module):
    
    def __init__(
        self, 
        num_ifos: int,
        c_depth: int=8, 
        n_chann: int=64, 
        l1: int=1024
        # lx: int=200
    ):
        
        super(WaveletCNNAE_xc, self).__init__()
        
        self.c_depth = c_depth
        self.n_chann = n_chann
        
        self.cap_norm = nn.GroupNorm(num_ifos, num_ifos)
        
        self.Conv_In_encode = nn.Conv1d(
                in_channels=num_ifos, 
                out_channels=self.n_chann, 
                kernel_size=1
            )
        
        self.Conv_Out_encode = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=1, 
                kernel_size=1
            )
        
        self.Conv_In_decode = nn.Conv1d(
                in_channels=1, 
                out_channels=self.n_chann, 
                kernel_size=1
            )
        
        self.Conv_Out_decode = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=num_ifos, 
                kernel_size=1
            )
        
        self.body_norm_encode = nn.GroupNorm(4 ,n_chann)
        self.body_norm_decode = nn.GroupNorm(4 ,n_chann)
        self.end_norm_encode = nn.BatchNorm1d(1)
        self.end_norm_decode = nn.BatchNorm1d(1)
        
        self.WaveNet_layers_encode = nn.ModuleList()
        self.WaveNet_layers_decode = nn.ModuleList()
        self.WaveNet_layers_dp = nn.ModuleList()
        
        
        for i in range(self.c_depth):

            conv_layer = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=self.n_chann,
                kernel_size=2,
                dilation=2**i
            )
            
            self.WaveNet_layers_encode.append(conv_layer)
            
        for i in range(self.c_depth-1, -1, -1):

            conv_layer = nn.Conv1d(
                in_channels=self.n_chann, 
                out_channels=self.n_chann,
                kernel_size=2,
                dilation=2**i
            )
            
            self.WaveNet_layers_decode.append(conv_layer)
            self.WaveNet_layers_dp.append(nn.ZeroPad1d(2**i))
        
        
#         self.Padding_layer = nn.ZeroPad1d(2**c_depth - 1)
                
        # self.L1 = nn.Linear(8192-2**c_depth, l1)
        
        # Consider replacing other batch normalizatoin layers with other nor method
        # Because batch norm are baised by the population of the CCSN rate in one batch 
        # This may produce overfitting model and will not be able to found at test phase
        # Question: Will we be able to figure out the side effect at infereceing phase?
                
#         self.conv_norm = nn.BatchNorm1d(200-2**c_depth + 1)
        self.L1 = nn.Linear(200-2**c_depth + 1, l1)
        self.L1_norm = nn.BatchNorm1d(l1)
        self.L2 = nn.Linear(l1, 200-2**c_depth + 1)
        self.L2_norm = nn.BatchNorm1d(200-2**c_depth + 1)

        nn.init.kaiming_normal_(self.Conv_In_encode.weight)
        nn.init.kaiming_normal_(self.Conv_Out_encode.weight)
        nn.init.constant_(self.Conv_In_encode.bias, 0.001)
        nn.init.constant_(self.Conv_Out_encode.bias, 0.001)
        
        nn.init.kaiming_normal_(self.Conv_In_decode.weight)
        nn.init.kaiming_normal_(self.Conv_Out_decode.weight)
        nn.init.constant_(self.Conv_In_decode.bias, 0.001)
        nn.init.constant_(self.Conv_Out_decode.bias, 0.001)

        # Initialize all the convolutional layer in between
        for conv_layer in self.WaveNet_layers_encode:
            nn.init.kaiming_normal_(conv_layer.weight)
            nn.init.constant_(conv_layer.bias, 0.001)
            
        for conv_layer in self.WaveNet_layers_decode:
            nn.init.kaiming_normal_(conv_layer.weight)
            nn.init.constant_(conv_layer.bias, 0.001)    

        nn.init.kaiming_uniform_(self.L1.weight)
        nn.init.kaiming_uniform_(self.L2.weight)
        nn.init.constant_(self.L1.bias, 0.001)
        nn.init.constant_(self.L2.bias, 0.001)
        
    def encode(self, x):
        
        x = self.cap_norm(x)
        x = self.Conv_In_encode(x)
        x = F.relu(x)
        
        # x = self.norm(x)
        
        for what_are_u_wavin_at in self.WaveNet_layers_encode:
            x = self.body_norm_encode(x)
            x = what_are_u_wavin_at(x)
            x = F.relu(x)
            
        x = self.Conv_Out_encode(x)
        x = F.relu(x)
        x = self.end_norm_encode(x)
        
        # print(x.shape)
        x = torch.flatten(x, 1)
        x = self.L1_norm(F.relu(self.L1(x)))
        
        # print('Encoder done')
        
        return x
    
    def decode(self, x):
        x = self.L2_norm(F.relu(self.L2(x)))
        
#         x = self.Padding_layer(x)
        
        x = torch.unsqueeze(x,1)

        # print(x.shape)
        
        # x = self.cap_norm(x)
        x = self.Conv_In_decode(x)
        x = F.relu(x)
        
        # x = self.norm(x)
        
        for (pad, dcd) in zip(self.WaveNet_layers_dp, self.WaveNet_layers_decode):
            # print(x.shape)
            x = self.body_norm_decode(x)
            x = pad(x)
            x = torch.flip(dcd(torch.flip(x, [-1])), [-1])
            x = F.relu(x)
        
        # print(x.shape)
        # print('CNN done')
        
        x = self.Conv_Out_decode(x)
        # print(x.shape)
        x = F.tanh(x)
        # print(x.shape)
        # x = self.end_norm_decode(x)
        
        # x = torch.flatten(x, 1)
        
        
        return x
    
    def forward(self,x):
        return self.decode(self.encode(x))

In [ ]:
def trainAE(dataset0_to_be_examined, cutID, version, datatype):
    
    if len(dataset0_to_be_examined) > 10 * trainable_params_AE:
        dataset = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_AE, replace = False)]
    else:
        dataset = dataset0_to_be_examined

    print('{} events passed to AE for training. '.format(len(dataset)))

    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainAE_Onedetector(dataset0_to_be_examined, cutID, version, datatype):
    
    if len(dataset0_to_be_examined) > 10 * trainable_params_AE:
        dataset = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_AE, replace = False)]
    else:
        dataset = dataset0_to_be_examined

    print('{} events passed to AE for training. '.format(len(dataset)))

    nTotal = len(dataset);
    nTrain = int(rTrain * nTotal)
    nTest = int(rTest * nTotal)

    X_train = dataset[:nTrain]
    X_test = dataset[-nTest:]
    X_validation = dataset[nTrain:-nTest]

    trainData = torch.FloatTensor(X_train)
    testData = torch.FloatTensor(X_test)
    validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(trainData)
    test_dataset = TensorDataset(testData)
    validation_dataset = TensorDataset(validationData)

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True)

    autoencoder = AutoEncoder_1det().cuda()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.00005)
    loss_func = nn.MSELoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):

        autoencoder.train()
        for batchidx, x in enumerate(trainDataLoader):
            x = x[0].cuda()
            encoded, decoded = autoencoder(x)
            loss_overall = loss_func(decoded, x)
            weighted_lossTrain = loss_overall

            optimizer.zero_grad()
            weighted_lossTrain.backward()
            optimizer.step()
            
        autoencoder.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, x in enumerate(validationDataLoader):
                x = x[0].cuda()
                encoded, decoded = autoencoder(x)
                lossVal = loss_func(decoded, x)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = weighted_lossTrain.item()
        loss_validation[epoch] = val_loss
    
    autoencoder.cpu().eval()
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    
    dcd_train = autoencoder(torch.FloatTensor(X_train))[1].detach().numpy()
    err_train = np.var(X_train-dcd_train, axis=1)
    dcd_test = autoencoder(torch.FloatTensor(X_test))[1].detach().numpy()
    err_test = np.var(X_test-dcd_test, axis=1)
    foo = ax[1].hist(err_train, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    foo = ax[1].hist(err_test, range=(0, max(err_train)), bins=50, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_AE_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
            
    return autoencoder.cpu().eval()

In [ ]:
def trainWSC(dataset0_to_be_examined, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    print('{} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1)))

    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

In [ ]:
def trainWSC_Onedetector(dataset0_to_be_examined, dataset1, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    print('{} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1)))

    nTotal0, nTotal1 = len(dataset0), len(dataset1);
    nTrain0, nTrain1 = int(rTrain * nTotal0), int(rTrain * nTotal1)
    nTest0 , nTest1  = int(rTest * nTotal0) , int(rTest * nTotal1)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1))))
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1))))
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1))))

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.FloatTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier_Onedetector().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([nTrain0/nTrain1])).cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_train))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Sigmoid()(wsc(torch.FloatTensor(X_test ))).detach().numpy().flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

In [ ]:
def trainWSC_3class(dataset0_to_be_examined, dataset1_to_be_examined, dataset2, cutID, version, datatype):
# dataset0: bkg set from AE
# dataset1: identified signal from AE

    if len(dataset0_to_be_examined) > 10 * trainable_params_WSC:
        dataset0 = dataset0_to_be_examined[np.random.choice(len(dataset0_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset0 = dataset0_to_be_examined
    
    if len(dataset1_to_be_examined) > 10 * trainable_params_WSC:
        dataset1 = dataset1_to_be_examined[np.random.choice(len(dataset1_to_be_examined), 10 * trainable_params_WSC, replace = False)]
    else:
        dataset1 = dataset1_to_be_examined

    print('{}, {} noise events and {} signal events passed to WSC for training. '.format(len(dataset0), len(dataset1), len(dataset2)))

    nTotal0, nTotal1, nTotal2 = len(dataset0), len(dataset1), len(dataset2);
    nTrain0, nTrain1, nTrain2 = int(rTrain * nTotal0), int(rTrain * nTotal1), int(rTrain * nTotal2)
    nTest0 , nTest1, nTest2  = int(rTest * nTotal0) , int(rTest * nTotal1), int(rTest * nTotal2)

    X_train = np.concatenate((dataset0[:nTrain0], dataset1[:nTrain1], dataset2[:nTrain2]))
    X_test = np.concatenate((dataset0[-nTest0:], dataset1[-nTest1:], dataset2[-nTest2:]))
    X_validation = np.concatenate((dataset0[nTrain0:-nTest0], dataset1[nTrain1:-nTest1], dataset2[nTrain2:-nTest2]))
    
    Y_train = np.concatenate((np.zeros((nTrain0, 1)), np.ones((nTrain1, 1)), 2 * np.ones((nTrain2, 1)))).flatten().astype(int)
    Y_test = np.concatenate((np.zeros((nTest0, 1)), np.ones((nTest1, 1)), 2 * np.ones((nTest2, 1)))).flatten().astype(int)
    Y_validation = np.concatenate((np.zeros((dataset0[nTrain0:-nTest0].shape[0], 1)), np.ones((dataset1[nTrain1:-nTest1].shape[0], 1)), 2 * np.ones((dataset2[nTrain2:-nTest2].shape[0], 1)))).flatten().astype(int)

#     trainData = torch.FloatTensor(X_train)
#     testData = torch.FloatTensor(X_test)
#     validationData = torch.FloatTensor(X_validation)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(Y_train))
    validation_dataset = TensorDataset(torch.FloatTensor(X_validation), torch.LongTensor(Y_validation))
#     train_dataset = TensorDataset(torch.FloatTensor(X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))), torch.FloatTensor(Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))))
#     validation_dataset = TensorDataset(torch.FloatTensor(X_validation.reshape((X_validation.shape[0], 1, X_validation.shape[1]))), torch.FloatTensor(Y_validation.reshape((Y_validation.shape[0], 1, Y_validation.shape[1]))))

    trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    validationDataLoader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle = True, drop_last=True)

    wsc = WSClassifier_3class().cuda()
    optimizer = optim.Adam(wsc.parameters(), lr=0.00005)
    loss_func = nn.CrossEntropyLoss().cuda()
    
    loss_train = np.empty(epochs)
    loss_validation = np.empty(epochs)

    for epoch in range(epochs):
#         t0 = time.time()
        wsc.train()
        for batchidx, (x, y) in enumerate(trainDataLoader):
            x = x.cuda()
            y = y.cuda()
            yprime = wsc(x)
            loss = loss_func(yprime, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        wsc.eval()
        with torch.no_grad():
            val_loss = 0
            for batchidx, (x, y) in enumerate(validationDataLoader):
                x = x.cuda()
                y = y.cuda()
                yprime = wsc(x)
                lossVal = loss_func(yprime, y)
                val_loss += lossVal.item()

            val_loss /= len(validationDataLoader)

        loss_train[epoch] = loss.item()
        loss_validation[epoch] = val_loss
#         print(time.time() - t0)
        
    wsc.cpu().eval()
    
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(loss_train)
    ax[0].plot(loss_validation)
    foo = ax[1].hist(nn.Softmax(dim=1)(wsc(torch.FloatTensor(X_train))).detach().numpy()[:,2].flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    foo = ax[1].hist(nn.Softmax(dim=1)(wsc(torch.FloatTensor(X_test ))).detach().numpy()[:,2].flatten(), range=(0, 1), bins=20, density=True, histtype="step")
    
    plt.savefig("../Pic_cached/SequentialTraining/WSL/training_WSC_"+cutID+"_" + version + "_" + datatype +"_trained.jpg")
    plt.close()
    
    return wsc.cpu().eval()

#### Pass the first glith double WSL (pretrained)

#### Train the noise AE

#### Train the noise WSL

#### Train the BBH CNN

#### Train the BBH CNN WSL

#### Finally, let the test sample (WSL training sample) pass the procedure to get the label

#### Final label WSL

### Full pipeline

In [ ]:
version = 'v12'

#### First assume the CNN AE and the WSL is pretrained

In [ ]:
Ncut = 5;
cutList = {};

max_glitch_L = 0.0024;
min_glitch_L = 0.0008;
cutList["glitch_L"] = np.linspace(min_glitch_L, max_glitch_L, Ncut);

max_glitch_H = 0.0024;
min_glitch_H = 0.0004;
cutList["glitch_H"] = np.linspace(min_glitch_H, max_glitch_H, Ncut);

max_bkg = 0.0018;
min_bkg = 0.0008;
cutList["noise"] = np.linspace(min_bkg, max_bkg, Ncut);


max_bbh_cnn = 0.0056
min_bbh_cnn = 0.0045
cutList['bbh_CNN'] = np.linspace(min_bbh_cnn, max_bbh_cnn, Ncut)

In [ ]:
# models = {}

In [ ]:
ind2datatype

In [ ]:
import itertools

In [ ]:
data_filtered

In [ ]:
version = 'v17'

In [ ]:
models

In [ ]:
cnt = 0;

ic = np.zeros(3, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1], ic[2] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    # if cnt < 23:
    #     continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf', 'glitch_oneglitch_onenoise']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    

    # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    # dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_H']
    
    # --- Train the glitch AE for L

    
    # models = {};
    models["glitch_L"] = torch.load('../Model_cached/1_det_L_glitch_trained_with_permutation_1.pt')
    models["glitch_H"] = torch.load('../Model_cached/1_det_H_glitch_trained_with_permutation_1.pt')

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = models['glitch_L'](torch.FloatTensor(data_filtered[dt][:,:101]))[1].detach().numpy();
        dcd_H = models['glitch_H'](torch.FloatTensor(data_filtered[dt][:,101:]))[1].detach().numpy();

        err_L = np.var(dcd_H - data_filtered[dt][:,:101], axis = -1).flatten()
        err_H = np.var(dcd_H - data_filtered[dt][:,101:], axis = -1).flatten()

        data_filtered[dt] = data_filtered[dt][np.logical_and(err_L > cutList['glitch_L'][ic[0]], err_H > cutList['glitch_H'][ic[1]])]
    
    # dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    # dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    # dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    # dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    if cnt != 1:
        
        print('Start noise AE')

        print('Start training the noise AE, dataset 0 size {}'.format(len(data_filtered['noise'])))

        currentStep = 'noise';
        model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
        models[currentStep] = model;   
        
        print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
        
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

#     print('Start noise WSL')

#     iPrev = 1

#     previousStep = 'noise';
#     modelPrev = models[previousStep]; # previous step AE
    
#     # train the WSC according to previous AE's cut
    
#     dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
#     # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#     # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
#     dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
#     dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
#     print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

#     model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
#     models[previousStep+"_WSC"] = model;

#     # --- Here I just filter the noise sample ---

#     data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
#     signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
#     print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

#     # data_wsl_noiselike = {}

#     # filter the data according to previous WSC
#     # for j in range(iPrev, 4):
#     #     dt = ind2datatype[j];
#     #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#     #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
#     #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

#     # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
#     # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#     # --- BBH CNN+WSL ---
#     # Here's a cut scan for BBH CNN

#     print('Start BBHCNN WSL')

#     models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

#     # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

#     dataset0 = data_noise_noiselike

#     dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
#     dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

#     print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
#     model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
#     models['BBH_CNN_WSC'] = model

#     print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}
    err = {}

    dcd['glitch_L'] = (models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101])))[1].detach().numpy()
    dcd['glitch_H'] = (models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:])))[1].detach().numpy()

    err['glitch_L'] = np.var(dcd['glitch_L'] - dataset_wsl_fft_collected[:,:101], axis = -1).flatten()
    err['glitch_H'] = np.var(dcd['glitch_H'] - dataset_wsl_fft_collected[:,101:], axis = -1).flatten()


    dcd['noise'] = (models['noise'](torch.FloatTensor(dataset_wsl_fft_collected)))[1].detach().numpy()
    err['noise'] = np.var(dcd['noise'] - dataset_wsl_fft_collected, axis = -1).flatten()

    # for dt in ['noise_WSC', 'BBH_CNN_WSC']:
    #     dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(err['glitch_L'] > cutList['glitch_L'][ic[0]], err['glitch_H'] > cutList['glitch_H'][ic[1]])

    noise_pass = err['noise'] > cutList['noise'][ic[2]]

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = dataset_ae_fft['glitch_oneglitch_onenoise']

    dataset1 = dataset_ae_fft['noise']

    dataset2 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, {}, dataset 2 size {}'.format(len(dataset0), len(dataset1), len(dataset2)))

    model = trainWSC_3class(dataset0, dataset1, dataset2, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
np.logical_not(np.array([True, False]))

In [ ]:
version = '15'

In [ ]:
cnt = 0;

ic = np.zeros(2, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    if not np.all(ic == np.array([4,0])):
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    

        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']

    model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_L']
    model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(['1','0']) + '_v2.json')['glitch_H']
    
    # --- Train the glitch AE for L

    models[previousStep+"_WSC_L"] = model_L
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L >= 0, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L >= 0, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L >= 0, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
ic

In [ ]:
not np.all(ic == np.array([4,0,1,0]))

In [ ]:
cnt = 0;

ic = np.zeros(4, dtype="int")

# loop for only the cut in glitch, noise and bbh as it's not really meaningful to set cut in sg w/o new signals
# ic[3] = Ncut-1;
# ic[4] = Ncut-1;

# listResult = {};
# listResult["cut"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)), dtype="int");
# listResult["ans"] = np.empty((Ncut**(len(list_datatype)-1), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype)-1), len(list_datatype)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype)-1), 2))

for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
# for ic[0], ic[1], ic[2], ic[3] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):
    cnt += 1;
    
    print(ic)
    
    if not np.all(ic == np.array([4,0,1,0])):
        continue
    # elif cnt > 85:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_fft_filtered = dataset_wsl_fft_collected
    dataset_wsl_filtered = dataset_wsl_collected
    
    cutID = "".join(str(ic[j]) for j in range(2)) + "_"+version
    
    
    print('Starting training on ic = {}'.format(''.join(str(ic[j] for j in range(4)))))
    
        # --- Glitch AE+WSL ---

    iPrev = 0
    
    previousStep = 'glitch';
    # modelPrev_L = models['glitch_L']; # previous step AE
    # modelPrev_H = models['glitch_H']; # previous step AE
    # train the WSC according to previous AE's cut
    
    # dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd_L = modelPrev_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy();
    # dcd_H = modelPrev_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy();
    # dataset1 = dataset_wsl_fft_filtered[np.logical_and(np.var(dataset_wsl_fft_filtered[:,:101]-dcd_L, axis=1) >= cutList['glitch_L'][ic[0]],
    #                                                np.var(dataset_wsl_fft_filtered[:,101:]-dcd_H, axis=1) >= cutList['glitch_H'][ic[1]])]
    
    # model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    # models[previousStep+"_WSC"] = model;

    # model_L = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_L']
    # model_H = torch.load('../Model_cached/WSL_Chia-Jui_glitches/two_WSL_for_Chia-Jui_noise_cutscheme' + ''.join(str(ic[j]) for j in range(2)) + '_v2.json')['glitch_H']
    
    model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], cutID, version, 'glitch')
    models['glitch_L'] = model
    
    model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,101:], cutID, version, 'glitch')
    models['glitch_H'] = model
    
    dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()
    
    dataset0 = data_filtered['glitch'][:,:101]
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_L'][ic[2]]][:,:101]
    
    model_L = trainWSC_Onedetector(dataset0, dataset1, cutID, version, 'glitch')
    models[previousStep+"_WSC_L"] = model_L
    
    
    dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy()
    
    dataset0 = data_filtered['glitch'][:,101:]
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,101:]-dcd, axis=1) > cutList['glitch_H'][ic[3]]][:,101:]
    
    model_H = trainWSC_Onedetector(dataset0, dataset1, cutID, version, 'glitch')
    models[previousStep+"_WSC_H"] = model_H

    
    # filter the data according to previous WSC
    for dt in ['glitch', 'noise', 'bbh', 'sglf', 'sghf']:
        dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(data_filtered[dt][:,:101]))).detach().numpy().flatten();
        dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(data_filtered[dt][:,101:]))).detach().numpy().flatten();
        data_filtered[dt] = data_filtered[dt][np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    
    dcd_L = nn.Sigmoid()(model_L(torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))).detach().numpy().flatten();
    dcd_H = nn.Sigmoid()(model_H(torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))).detach().numpy().flatten();

    dataset_wsl_fft_filtered = dataset_wsl_fft_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]
    dataset_wsl_filtered = dataset_wsl_filtered[np.logical_and(dcd_L > 0.5, dcd_H > 0.5)]

#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
    
    # train the current step AE

    print('Start noise AE')

    print('Start training the noise WSL, dataset 0 size {}'.format(len(data_filtered['noise'])))

    currentStep = 'noise';
    model = trainAE(data_filtered[currentStep], cutID, version, currentStep);
    models[currentStep] = model;   
    
    print('Noise AE trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Noise AE+WSL ---
    # Here's a cut scan for the noise AE

    print('Start noise WSL')

    iPrev = 1

    previousStep = 'noise';
    modelPrev = models[previousStep]; # previous step AE
    
    # train the WSC according to previous AE's cut
    
    dataset0 = data_filtered[previousStep] # here they haven't been updated yet
    
    # dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
    # dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    dcd = modelPrev(torch.FloatTensor(dataset_wsl_fft_filtered))[1].detach().numpy();
    dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered-dcd, axis=1) >= cutList[previousStep][ic[0]]]
    
    print('Start training the noise WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, previousStep)
    models[previousStep+"_WSC"] = model;

    # --- Here I just filter the noise sample ---

    data_noise_noiselike = data_filtered['noise'][nn.Sigmoid()(model(torch.FloatTensor(data_filtered['noise']))).detach().numpy().flatten() <= 0.5]
    signal_noise_like_args = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft_filtered))).detach().numpy().flatten() <= 0.5
    
    print('Noise WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    # data_wsl_noiselike = {}

    # filter the data according to previous WSC
    # for j in range(iPrev, 4):
    #     dt = ind2datatype[j];
    #     dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
    #     data_filtered[dt] = data_filtered[dt][dcd>0.5]
    #     data_noiselike[dt] = data_filtered[dt][dcd<=0.5]

    # dcd = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten()
    # data_wsl_noiselike = dataset_wsl_collected[dcd <= 0.5].reshape(-1,2,200)
    
#         # filter the data
#         for j in range(iPrev+1, 4):
#             dt = ind2datatype[j];
#             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
#             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
    # --- BBH CNN+WSL ---
    # Here's a cut scan for BBH CNN

    print('Start BBHCNN WSL')

    models['BBH_CNN'] = torch.load('../Model_cached/CNN_BBH/model_dep_3_chnl_4_btn_20_v4.pt')

    # Note that for CNN, we have to use the initial input data (2 * 200 in timeseries)

    dataset0 = data_noise_noiselike

    dcd = models['BBH_CNN'](torch.FloatTensor(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200))).detach().numpy()
    dataset1 = dataset_wsl_fft_filtered[signal_noise_like_args][np.mean(np.var(dataset_wsl_filtered[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]]

    print('Start training the BBHCNN WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))
    
    model = trainWSC(dataset0, dataset1, cutID, version, 'BBH_CNN')
    models['BBH_CNN_WSC'] = model

    print('BBHCNN WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))
    
    # --- Finalized WSL ---

    print('Start final WSL for cnt = {}'.format(cnt))

    dcd = {}

    dcd['glitch_WSC_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(dataset_wsl_fft_collected[:,:101]))).detach().numpy().flatten()
    dcd['glitch_WSC_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(dataset_wsl_fft_collected[:,101:]))).detach().numpy().flatten()


    for dt in ['noise_WSC', 'BBH_CNN_WSC']:
        dcd[dt] = nn.Sigmoid()(models[dt](torch.FloatTensor(dataset_wsl_fft_collected))).detach().numpy().flatten()

    glitch_pass = np.logical_and(dcd['glitch_WSC_L'] > 0.5, dcd['glitch_WSC_H'] > 0.5)

    noise_pass = np.logical_or(dcd['noise_WSC'] > 0.5, np.logical_and(dcd['noise_WSC'] <= 0.5, dcd['BBH_CNN_WSC'] > 0.5))

    passed_args = np.logical_and(glitch_pass, noise_pass)

    # dataset0 = np.concatenate((dataset_ae_fft['glitch'], dataset_ae_fft['noise']), axis = 0)

    dataset0 = data_filtered['noise']

    dataset1 = dataset_wsl_fft_collected[passed_args]

    print('Start training the final WSL, dataset 0 size {}, dataset 1 size {}'.format(len(dataset0), len(dataset1)))

    model = trainWSC(dataset0, dataset1, cutID, version, 'Final')

    models['Final_WSC'] = model


    print('Final WSL trained completed, this epoch is using time {}'.format(str(time.time() - t0)))

    torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    print(models.keys())
    




#     dcd = {};
#     err = {};
#     ans = np.zeros(len(testset), dtype="int")
    
#     for datatype in list_datatype:
#         dcd[datatype] = models[datatype](torch.FloatTensor(testset))[1].detach().numpy()
#         err[datatype] = np.var(testset-dcd[datatype], axis=1)
        
#     not_select = np.array([True]*len(testset));

#     for iStep in range(len(list_datatype)):
#         datatype = ind2datatype[iStep];
#         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
#         ans[ind_pass] = iStep;
#         not_select[ind_pass] = False;
        
#     ans[not_select] = -1;
    
#     listResult["cut"][cnt] = ic;
#     listResult["ans"][cnt] = ans;
    
#     acc = np.zeros(len(ind2datatype));
    
#     for i in range(len(ind2datatype)):
#         acc[i] = np.sum(np.logical_and(ans==i, correct_ans==i))/Nsample[ind2datatype[i]];
        
#     listResult["accuracy_4"][cnt] = acc;
    
#     listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["glitch", "noise"])/np.sum(Nsample[dtype] for dtype in ["glitch", "noise"]), 
#                                      np.sum(acc[datatype2ind[dtype]]*Nsample[dtype] for dtype in ["bbh", "sg"])/np.sum(Nsample[dtype] for dtype in ["bbh", "sg"])]
    # cnt += 1
    print(cnt)
    print(time.time() - t0)
    
# listResult["total_accuracy"] = np.sum(listResult["ans"]==correct_ans, axis=1)/len(testset);
# torch.save(listResult, "../data/SequentialTraining/training_performance_"+version+".json")

In [ ]:
dataset_wsl_filtered.shape

In [ ]:
(np.mean(np.var(dataset_wsl_collected[signal_noise_like_args].reshape(-1,2,200)-dcd, axis = 2), axis = 1) <= cutList['bbh_CNN'][ic[1]]).shape

In [ ]:
signal_noise_like_args = np.argwhere(nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_filtered))).detach().numpy().flatten() <= 0.5)

In [ ]:
signal_noise_like_args

In [ ]:
dataset_wsl_collected[signal_noise_like_args].shape

### Evaluating the pipeline data

In [ ]:
dataset_wsl_fft_collected.shape

In [ ]:
sequence

In [ ]:
N_wsl.keys()

In [ ]:
correct_ans = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [2]*N_wsl['bbh'], [3]*N_wsl['sglf'], [4]*N_wsl['sghf']))
correct_ans_withoutsignal = np.hstack(([0]*N_wsl['glitch'], [1]*N_wsl['noise'], [2]*(N_wsl['bbh']+N_wsl['sglf']+N_wsl['sghf'])))

In [ ]:
correct_ans_withoutsignal.shape

In [ ]:
list_datatype_withoutsignal = ['glitch','noise']

In [ ]:
testset = dataset_wsl_fft_collected

In [ ]:
version = 'v17'

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**3, 3), dtype="int");
listResult["ans"] = np.empty((Ncut**3, len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0
ic_withoutsignal = np.zeros(3, dtype = int)

listResult["FPR"] = np.empty((Ncut**3,1))


for ic_withoutsignal[0], ic_withoutsignal[1], ic_withoutsignal[2] in itertools.product(np.arange(Ncut), np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(3)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    # dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    # dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    # dcd['noise_WSC'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    # dcd['Final_WSC'] = nn.Sigmoid()(models["Final_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    dcd['Final_WSC'] = nn.Softmax(dim=1)(models["Final_WSC"](torch.FloatTensor(testset)))[:,2].detach().numpy()
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    # datatype = 'glitch'
    # ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] <= 0.5, dcd['glitch_H'] <= 0.5))
    # ans[ind_pass] = 0;
    # not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    # datatype = 'noise_WSC'
    datatype = 'Final_WSC'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
    listResult['ans'][cnt] = ans
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
version

In [ ]:
listResult = {};
listResult["cut"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)), dtype="int");
listResult["ans"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(testset)), dtype="int");
# listResult["accuracy_4"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), len(list_datatype_withoutsignal)))
# listResult["accuracy_2"] = np.empty((Ncut**(len(list_datatype_withoutsignal)), 2))
cnt = 0
ic_withoutsignal = np.zeros(2, dtype = int)

listResult["FPR"] = np.empty((Ncut**(len(list_datatype_withoutsignal)),1))


for ic_withoutsignal[0], ic_withoutsignal[1] in itertools.product(np.arange(Ncut), np.arange(Ncut)):

    
    # if cnt < 86:
    #     continue
    
    if not np.all(ic_withoutsignal == np.array([4,0])):
        continue
    
    t0 = time.time()
    data_filtered = {};
    for dt in sequence:
        data_filtered[dt] = dataset_ae_fft[dt]
#     data_filtered["noise"] = bkg_fft;
#     data_filtered["bbh"] = bbh_fft;
#     data_filtered["sg"] = sg_fft;

    dataset_wsl_filtered = dataset_wsl_fft_collected
        
#     for iPrev in range(3):
#         previousStep = ind2datatype[iPrev];
#         modelPrev = models[previousStep]; # previous step AE
        
#         # train the WSC according to previous AE's cut
        
#         dataset0 = data_filtered[previousStep] # here they haven't been updated yet
        
#         dcd = modelPrev(torch.FloatTensor(dataset0))[1].detach().numpy();
#         dataset1 = dataset0[np.var(dataset0-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         dcd = modelPrev(torch.FloatTensor(dataset_wsl_filtered))[1].detach().numpy();
#         dataset1 = dataset_wsl_filtered[np.var(dataset_wsl_filtered-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]
        
#         model = trainWSC(dataset0, dataset1, cutID)
#         models[previousStep+"_WSC"] = model;
        
#         # filter the data according to previous WSC
#         for j in range(iPrev, 4):
#             dt = ind2datatype[j];
#             dcd = nn.Sigmoid()(model(torch.FloatTensor(data_filtered[dt]))).detach().numpy().flatten();
#             data_filtered[dt] = data_filtered[dt][dcd>0.5]
        
# #         # filter the data
# #         for j in range(iPrev+1, 4):
# #             dt = ind2datatype[j];
# #             dcd = modelPrev(torch.FloatTensor(data_filtered[dt]))[1].detach().numpy()
# #             data_filtered[dt] = data_filtered[dt][np.var(data_filtered[dt]-dcd, axis=1) >= cutList[previousStep][ic[iPrev]]]            
        
#         # train the current step AE
#         currentStep = ind2datatype[iPrev+1];
#         model = trainAE(data_filtered[currentStep], cutID);
#         models[currentStep] = model;
        
#     torch.save(models, "../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(3)) + "_"+version+".json")
#     print(models.keys())
    models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic_withoutsignal[j]) for j in range(2)) + "_"+version+".json")
    # print("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(str(ic[j]) for j in range(2)) + "_"+version+".json")
    dcd = {};
    err = {};
    ans = np.zeros(len(testset), dtype="int")
    
    dcd['glitch_L'] = nn.Sigmoid()(models['glitch_WSC_L'](torch.FloatTensor(testset[:,:101]))).detach().numpy().reshape(-1)
    dcd['glitch_H'] = nn.Sigmoid()(models['glitch_WSC_H'](torch.FloatTensor(testset[:,101:]))).detach().numpy().reshape(-1)
    dcd['noise_WSC'] = nn.Sigmoid()(models["noise_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    dcd['Final_WSC'] = nn.Sigmoid()(models["Final_WSC"](torch.FloatTensor(testset))).detach().numpy().reshape(-1)
    
    not_select = np.array([True]*len(testset));

    # for iStep in range(len(list_datatype_withoutsignal)):
    #     datatype = ind2datatype[iStep];
    #     if datatype == 'sg':
    #         ind_pass = np.logical_and(not_select, err[datatype] <= cutList[datatype][ic[iStep]]);
    #     else:
    #         ind_pass = np.logical_and(not_select, dcd[datatype] <= 0.5);
    #     ans[ind_pass] = iStep;
    #     not_select[ind_pass] = False;
        
    # Pass glitch first
    
    datatype = 'glitch'
    ind_pass = np.logical_and(not_select, np.logical_or(dcd['glitch_L'] < 0, dcd['glitch_H'] <= 0.5))
    ans[ind_pass] = 0;
    not_select[ind_pass] = False;
    # print(dcd['glitch'])
    
    # Leftover are noise and signals
    
    # datatype = 'noise_WSC'
    datatype = 'Final_WSC'
    ind_pass = np.logical_and(not_select, dcd[datatype] <= np.sort(dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]])[int(0.1 * (np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])))])
    # ind_pass = np.logical_and(not_select, err[datatype] <= np.sort(err[datatype][-N_wsl['bbh']-N_wsl['sg']:])[int(0.1 * (N_wsl['bbh']+N_wsl['sg']))])
    noise_number = np.sum(np.logical_and(not_select, correct_ans_withoutsignal == 1))
    passed_noise_number = noise_number - np.sum(np.logical_and(ind_pass, correct_ans_withoutsignal == 1))
    ans[ind_pass] = 1;
    not_select[ind_pass] = False;
    
    ans[not_select] = -1
    
    FPR = passed_noise_number / noise_number
    
    # print(dcd['noise'])
    print('For cnt = {}, totally {} noise events passed the glitch WSL, and {} noise events within the threshold for TPR=0.9'.format(cnt, noise_number, passed_noise_number))
    listResult['FPR'][cnt] = FPR
    listResult['cut'][cnt] = ic_withoutsignal
    listResult['ans'][cnt] = ans
        
    # ans[not_select] = -1;

    # listResult["cut"][cnt] = ic_withoutsignal;
    # listResult["ans"][cnt] = ans;

    # acc = np.zeros(len(ind2datatype));

    # for i in range(len(ind2datatype)):
    #     acc[i] = np.sum(np.logical_and(ans==i, correct_ans_withoutsignal==i))/N_wsl[ind2datatype[i]];
        
    # listResult["accuracy_4"][cnt] = acc;

    # listResult["accuracy_2"][cnt] = [ np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["glitch", "noise"])/np.sum(N_wsl[dtype] for dtype in ["glitch", "noise"]), 
    #                                     np.sum(acc[datatype2ind[dtype]]*N_wsl[dtype] for dtype in ["bbh", "sg"])/np.sum(N_wsl[dtype] for dtype in ["bbh", "sg"])]

    cnt += 1;

In [ ]:
N_wsl['noise']

In [ ]:
np.min(listResult['FPR'])

In [ ]:
listResult['cut'][29]

In [ ]:
model = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "104" + "_"+version+".json")['Final_WSC']

In [ ]:
model(torch.FloatTensor(dataset_wsl_fft['noise'])).shape

In [ ]:
score_bkg = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft['noise'])))[:,2].detach().numpy()
score_glitch = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft['glitch'])))[:,2].detach().numpy()

for type in ['bbh', 'sglf', 'sghf']:
    
    plt.hist(score_bkg, bins = 50, range = (0,1), histtype = 'step', density = True, label = 'noise')
    plt.hist(score_glitch, bins = 50, range = (0,1), histtype = 'step', density = True, label = 'glitch')
    
    for snr in ['5-12', '12-24', '24-48', '48-96']:
        
        score = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft[type+snr])))[:,2].detach().numpy()
        plt.hist(score, bins = 50, range = (0,1), histtype = 'step', density = True, label = type+snr)
        


    plt.legend()
    plt.show()

In [ ]:
FPR_list = {}

score_bkg = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft['noise'])))[:,2].detach().numpy()
score_glitch = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft['glitch'])))[:,2].detach().numpy()

for type in ['bbh', 'sglf', 'sghf']:
    
    for snr in ['5-12', '12-24', '24-48', '48-96']:
        
        score = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft[type+snr])))[:,2].detach().numpy()
        
        noise_events_passed = np.sum(score_bkg > np.sort(score)[int(0.1 * len(score))])
        glitch_events_passed = np.sum(score_glitch > np.sort(score)[int(0.1 * len(score))])
        
        FPR = (noise_events_passed + glitch_events_passed) / (len(score_bkg) + len(score_glitch))
        
        FPR_list[type+snr] = FPR

In [ ]:
FPR_list

In [ ]:
snr_list = ['5-12', '12-24', '24-48', '48-96']

plt.figure(figsize=(8,6))

for type in ['bbh', 'sglf', 'sghf']:
    
    FPR_data = np.empty(4)
    
    for i in range(4):
        
        FPR_data[i] = FPR_list[type+snr_list[i]]
        
    central_point = np.array([9,18,36,72])
        
    # plt.scatter(['5-12','12-24','24-48','48-96'], FPR_data, label = type)
    plt.plot(['5-12','12-24','24-48','48-96'], FPR_data, marker = 'o', label = type)
    
    
# plt.xlim(0,96)
plt.ylim(-0.1,1.1)
plt.xlabel('SNR range')
plt.ylabel('FPR')

plt.title('FPR distribution versus SNR, TPR fixed at 0.9')

plt.legend()
plt.show()

In [ ]:
TPR_list = {}

score_bkg = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft['noise'])))[:,2].detach().numpy()
score_glitch = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft['glitch'])))[:,2].detach().numpy()

score_bkgevents = np.concatenate((score_bkg, score_glitch))

threshold = np.sort(score_bkgevents)[-int(0.001 * len(score_bkgevents))]

for type in ['bbh', 'sglf', 'sghf']:
    
    for snr in ['5-12', '12-24', '24-48', '48-96']:
        
        score = nn.Softmax(dim = 1)(model(torch.FloatTensor(dataset_wsl_fft[type+snr])))[:,2].detach().numpy()
        
        TPR = np.sum(score > threshold) / len(score)
        
        TPR_list[type+snr] = TPR

In [ ]:
TPR_list

In [ ]:
snr_list = ['5-12', '12-24', '24-48', '48-96']

plt.figure(figsize=(8,6))

for type in ['bbh', 'sglf', 'sghf']:
    
    TPR_data = np.empty(4)
    
    for i in range(4):
        
        TPR_data[i] = TPR_list[type+snr_list[i]]
        
    # central_point = np.array([9,18,36,72])
        
    # plt.scatter(['5-12','12-24','24-48','48-96'], FPR_data, label = type)
    plt.plot(['5-12','12-24','24-48','48-96'], TPR_data, marker = 'o', label = type)
    
    
# plt.xlim(0,96)
plt.ylim(-0.1,1.1)
plt.xlabel('SNR range')
plt.ylabel('TPR')

plt.title('TPR distribution versus SNR, FPR fixed at 0.001')

plt.legend()
plt.show()

In [ ]:
snr_list = ['5-12', '12-24', '24-48', '48-96']

plt.figure(figsize=(8,6))

for type in ['bbh', 'sglf', 'sghf']:
    
    TPR_data = np.empty(4)
    
    for i in range(4):
        
        TPR_data[i] = TPR_list[type+snr_list[i]]
        
    # central_point = np.array([9,18,36,72])
        
    # plt.scatter(['5-12','12-24','24-48','48-96'], FPR_data, label = type)
    plt.plot(['5-12','12-24','24-48','48-96'], TPR_data, marker = 'o', label = type)
    
    
# plt.xlim(0,96)
plt.ylim(-0.1,1.1)
plt.xlabel('SNR range')
plt.ylabel('TPR')

plt.title('TPR distribution versus SNR, FPR fixed at 0.01')

plt.legend()
plt.show()

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:] == 0)

In [ ]:
cnt = 20

np.sum(listResult['ans'][cnt][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:] == 1)

In [ ]:
cnt = 0

np.sum(listResult['ans'][cnt][N_wsl['glitch']:N_wsl['glitch']+N_wsl['noise']] == 0)

In [ ]:
dcd[datatype][-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:][not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:]].shape

In [ ]:
np.sum(not_select[-N_wsl['bbh']-N_wsl['sglf']-N_wsl['sghf']:])

In [ ]:
version = 'v12'

In [ ]:
models = torch.load("../Data_cached/SequentialTraining/WSL/trained_model" + "".join(['4','0']) + "_"+version+".json")

In [ ]:
dataset_wsl_fft.keys()

In [ ]:
model = models['noise_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['BBH_CNN_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v12

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v15

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
models

In [ ]:
# v14

model = models['Final_WSC']

bkg_filtered = dataset_wsl_fft['noise']
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
# glitch_filtered = dataset_wsl_fft['glitch']

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
# dcd_glitch = model(torch.FloatTensor(glitch_fft))[1].detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    plt.title("Performance of Final WSC")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v14

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# v14

model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
cutList['glitch_L']

In [ ]:
ic[2]

In [ ]:
# v14

model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'test')

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_L'][ic[2]], color="k", linestyle="--")
plt.legend()

In [ ]:
# v14

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_L'][ic[2]], color="k", linestyle="--")
plt.legend()

In [ ]:
# v14

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_H'][ic[3]], color="k", linestyle="--")
plt.legend()

### test training the glitch AE

In [ ]:
models = {}

In [ ]:
models = {};
models["glitch_L"] = torch.load('../Model_cached/1_det_L_glitch_trained_with_permutation_1.pt')
models["glitch_H"] = torch.load('../Model_cached/1_det_H_glitch_trained_with_permutation_1.pt')

In [ ]:
dataset_wsl_fft_filtered = dataset_wsl_fft_collected

In [ ]:
# model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,:101], 'test', 'test', 'glitch')
# models['glitch_L'] = model

# model = trainAE_Onedetector(dataset_ae_fft['glitch'][:,101:], 'test', 'test', 'glitch')
# models['glitch_H'] = model

dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft_filtered[:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,:101]-dcd, axis=1) > cutList['glitch_L'][1]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L


dcd = models['glitch_H'](torch.FloatTensor(dataset_wsl_fft_filtered[:,101:]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,101:]
dataset1 = dataset_wsl_fft_filtered[np.var(dataset_wsl_fft_filtered[:,101:]-dcd, axis=1) > cutList['glitch_H'][0]][:,101:]

model_H = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_H"] = model_H

In [ ]:
dcd = models['glitch_L'](torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()

dataset0 = dataset_ae_fft['glitch'][:,:101]
dataset1 = dataset_wsl_fft['noise'][np.var(dataset_wsl_fft['noise'][:,:101]-dcd, axis=1) > cutList['glitch_L'][3]][:,:101]

model_L = trainWSC_Onedetector(dataset0, dataset1, 'test', 'test', 'glitch')
models["glitch_WSC_L"] = model_L

In [ ]:
# test

model = models['glitch_L']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,:101]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,:101]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,:101]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,:101]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,:101]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,:101] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Livinston noise")
    plt.axvline(cutList['glitch_L'][0], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_WSC_L']

bkg_filtered = dataset_wsl_fft['noise'][:,:101]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,:101]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC L")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,:101]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()
    # plt.ylim(0,2000)

In [ ]:
# test

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
# foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
# foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
plt.title("trained with noise")
plt.axvline(cutList['glitch_H'][0], color="k", linestyle="--")
plt.legend()

In [ ]:
# test

model = models['glitch_H']

dcd_bkg = model(torch.FloatTensor(dataset_wsl_fft['noise'][:,101:]))[1].detach().numpy()
# dcd_bbh = model(torch.FloatTensor(bbh_filtered))[1].detach().numpy()
# dcd_sg = model(torch.FloatTensor(sg_filtered))[1].detach().numpy()
dcd_glitch = model(torch.FloatTensor(dataset_ae_fft['glitch'][:,101:]))[1].detach().numpy()

for key in ['bbh', 'sglf', 'sghf']:

    foo = plt.hist(np.var(dataset_wsl_fft['noise'][:,101:]-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    foo = plt.hist(np.var(dataset_ae_fft['glitch'][:,101:]-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")

    for snr in snr_range:
        key_to_plot = key+snr
        dcd_signal = model(torch.FloatTensor(dataset_wsl_fft[key_to_plot][:,101:]))[1].detach().numpy()
        plt.hist(np.var(dataset_wsl_fft[key_to_plot][:,101:] - dcd_signal, axis = 1), histtype="step", range=(0, 0.01), bins=50, density=True, label=key_to_plot)



    plt.title("trained with Hanford noise")
    plt.axvline(cutList['glitch_H'][1], color="k", linestyle="--")
    plt.legend()
    
    plt.show()

In [ ]:
# test
model = models['glitch_WSC_H']

bkg_filtered = dataset_wsl_fft['noise'][:,101:]
# bbh_filtered = dataset_wsl_fft['bbh']
# sg_filtered = dataset_wsl_fft['sg']
glitch_filtered = dataset_wsl_fft['glitch'][:,101:]

# c0[currentStep] = 0.0022;
    
dcd_bkg = nn.Sigmoid()(model(torch.FloatTensor(bkg_filtered))).detach().numpy()
# dcd_bbh = nn.Sigmoid()(model(torch.FloatTensor(bbh_filtered))).detach().numpy()
# dcd_sg = nn.Sigmoid()(model(torch.FloatTensor(sg_filtered))).detach().numpy()
dcd_glitch = model(torch.FloatTensor(glitch_filtered)).detach().numpy()

for signal_type in ['bbh', 'sglf', 'sghf']:

    # foo = plt.hist(np.var(bkg_filtered-dcd_bkg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="noise")
    # foo = plt.hist(np.var(bbh_filtered-dcd_bbh, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="bbh")
    # foo = plt.hist(np.var(sg_filtered-dcd_sg, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="sg")
    # foo = plt.hist(np.var(glitch_fft-dcd_glitch, axis=1), histtype="step", range=(0, 0.01), bins=50, density=True, label="glitch")
    foo = plt.hist(dcd_bkg, histtype="step", range=(0, 1), bins=50, density=True, label="noise")
    # foo = plt.hist(dcd_bbh, histtype="step", range=(0, 1), bins=50, density=True, label="bbh")
    # foo = plt.hist(dcd_sg, histtype="step", range=(0, 1), bins=50, density=True, label="sg")
    foo = plt.hist(dcd_glitch, histtype="step", range=(0, 1), bins=50, density=True, label="glitch")
    plt.title("Performance of Glitch WSC H")
        
        
    for snr in snr_range:

        key = signal_type+snr
        dcd_signal = nn.Sigmoid()(model(torch.FloatTensor(dataset_wsl_fft[key][:,101:]))).detach().numpy()
        foo = plt.hist(dcd_signal, histtype="step", range=(0, 1), bins=50, density=True, label=key)


    # plt.axvline(c0[currentStep], color="k", linestyle="--")
    # plt.axvline(np.sort(dcd_bbh.flatten())[int(0.1 * len(bbh_filtered))], color = 'k', linestyle = '--')
    plt.legend()
    plt.show()